# Analisis General

## Pre-procesamiento

In [9]:
import pandas as pd
import numpy as np

# Leer los tres archivos
arma_df = pd.read_excel("./datos/resultados_140_FINAL_ARMA.xlsx")
arima_df = pd.read_excel("./datos/resultados_140_ARIMA_FINAL.xlsx")
setar_df = pd.read_excel("./datos/resultados_140_SETAR_FINAL.xlsx")

# Filtrar los que no tienen "Promedio" en la columna "Paso"
arma_df = arma_df[arma_df['Paso'] != 'Promedio']
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
setar_df = setar_df[setar_df['Paso'] != 'Promedio']

# Lista de modelos (columnas a promediar)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario (de la columna del modelo)
    arma_promedio = arma_df[modelo].mean() if modelo in arma_df.columns else np.nan
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    setar_promedio = setar_df[modelo].mean() if modelo in setar_df.columns else np.nan
    
    fila['ARMA'] = arma_promedio
    fila['ARIMA'] = arima_promedio
    fila['SETAR'] = setar_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARMA': arma_promedio,
        'ARIMA': arima_promedio,
        'SETAR': setar_promedio
    }
    
    # Filtrar NaN si existen
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)

# Redondear valores para mejor visualización
columnas_numericas = ['ARMA', 'ARIMA', 'SETAR']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")

# Guardar tabla comparativa en Excel
tabla_comparativa.to_excel("Tabla_Comparativa_Modelos.xlsx", index=False)
print("Tabla comparativa guardada en 'Tabla_Comparativa_Modelos.xlsx'")

# Procesamiento especial para SETAR
if 'Descripción' in setar_df.columns:
    setar_df = setar_df.drop('Descripción', axis=1)

# Agregar columna ESCENARIO a cada DataFrame antes de concatenar
arma_df['ESCENARIO'] = 'Lineal - estacionario'
arima_df['ESCENARIO'] = 'Lineal - NO estacionario'
setar_df['ESCENARIO'] = 'NO lineal - estacionario'

# Concatenar los tres dataframes
base_consolidada = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Guardar en un archivo Excel
base_consolidada.to_excel("Base_140_3_escenarios.xlsx", index=False)

print("\nArchivo 'Base_140_3_escenarios.xlsx' creado exitosamente!")
print(f"\nTotal de filas: {len(base_consolidada)}")
print(f"- ARMA: {len(arma_df)} filas")
print(f"- ARIMA: {len(arima_df)} filas")
print(f"- SETAR: {len(setar_df)} filas")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARMA   ARIMA  SETAR Mejor_Escenario
              AREPD 0.9722  9.7604 0.6955           SETAR
            AV-MCPS 0.7103  3.0618 0.6706           SETAR
Block Bootstrapping 0.9172 10.9690 0.6460           SETAR
             DeepAR 0.5779  3.1462 0.6249            ARMA
         EnCQR-LSTM 1.0382  5.8306 0.8408           SETAR
               LSPM 0.7696  1.1140 0.6751           SETAR
              LSPMW 1.0604  3.5094 0.6868           SETAR
               MCPS 0.7222  2.8994 0.6916           SETAR
    Sieve Bootstrap 0.5547  0.5479 0.6304           ARIMA

Tabla comparativa guardada en 'Tabla_Comparativa_Modelos.xlsx'

Archivo 'Base_140_3_escenarios.xlsx' creado exitosamente!

Total de filas: 5040
- ARMA: 1680 filas
- ARIMA: 1680 filas
- SETAR: 1680 filas


## Analisis general

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from itertools import combinations
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./Base_140_3_escenarios.xlsx"
DIR_SALIDA = "./resultados_generales_robustos"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

COLORES_MODELOS = {
    'AREPD': '#e41a1c',
    'AV-MCPS': '#377eb8',
    'Block Bootstrapping': '#4daf4a',
    'DeepAR': '#984ea3',
    'EnCQR-LSTM': '#ff7f00',
    'LSPM': '#ffff33',
    'LSPMW': '#a65628',
    'MCPS': '#f781bf',
    'Sieve Bootstrap': '#999999'
}

CARACTERISTICAS_META_MODELO = [
    'Estacionario', 'Lineal', 'Tipo de Modelo',
    'Distribución', 'Varianza error', 'Paso'
]
CARACTERISTICAS_NUMERICAS_META_MODELO = ['Varianza error', 'Paso']
CARACTERISTICAS_CATEGORICAS_META_MODELO = [
    'Estacionario', 'Lineal', 'Tipo de Modelo', 'Distribución'
]

# ============================================================================
# FUNCIONES AUXILIARES - TEST DIEBOLD-MARIANO
# ============================================================================

def diebold_mariano_test(errores1, errores2, h=1, alternative='two-sided', 
                         loss_function='squared'):
    """
    CORRECCIÓN: Para ECRPS, usar 'none' como loss_function
    """
    e1 = np.asarray(errores1)
    e2 = np.asarray(errores2)
    
    if len(e1) != len(e2):
        raise ValueError("Los vectores de errores deben tener la misma longitud")
    
    n = len(e1)
    
    # ✅ CORRECCIÓN: Manejo correcto de funciones de pérdida
    if loss_function == 'squared':
        loss1 = e1 ** 2
        loss2 = e2 ** 2
    elif loss_function == 'absolute':
        loss1 = np.abs(e1)
        loss2 = np.abs(e2)
    elif loss_function == 'none':
        # Para ECRPS - ya son medidas de pérdida directas
        loss1 = e1
        loss2 = e2
    else:
        raise ValueError("loss_function debe ser 'squared', 'absolute' o 'none'")
    
    # Diferencia de pérdidas
    d = loss1 - loss2
    d_mean = np.mean(d)
    
    # ✅ CORRECCIÓN: Cálculo robusto de la varianza HAC
    # Para h=1 (pronóstico un paso adelante), no necesitamos corrección por autocorrelación
    if h == 1:
        var_d = np.var(d, ddof=1) / n
    else:
        # Corrección HAC para h > 1
        gamma_0 = np.var(d, ddof=1)
        gamma_sum = 0
        max_lags = min(h-1, n-1)  # No más lags que observaciones
        for k in range(1, max_lags + 1):
            if k < n:
                gamma_k = np.cov(d[:-k], d[k:], ddof=1)[0,1] if len(d) > k else 0
                gamma_sum += (1 - k/(max_lags+1)) * gamma_k  # Kernel de Bartlett
        
        var_d = (gamma_0 + 2 * gamma_sum) / n
    
    # ✅ CORRECCIÓN: Solo aplicar corrección HLN si h > 1
    if h > 1:
        hlnc = np.sqrt((n + 1 - 2 * h + h * (h - 1) / n) / n)
    else:
        hlnc = 1.0
    
    if var_d > 0:
        dm_stat = d_mean / np.sqrt(var_d)
        dm_stat_corrected = dm_stat * hlnc
    else:
        dm_stat = 0
        dm_stat_corrected = 0
    
    # Cálculo de p-value (usando distribución normal asintótica)
    if alternative == 'two-sided':
        p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat_corrected)))
    elif alternative == 'less':
        p_value = stats.norm.cdf(dm_stat_corrected)
    elif alternative == 'greater':
        p_value = 1 - stats.norm.cdf(dm_stat_corrected)
    else:
        raise ValueError("alternative debe ser 'two-sided', 'less' o 'greater'")

    return {
        'dm_statistic': dm_stat,
        'dm_statistic_corrected': dm_stat_corrected,
        'p_value': p_value,
        'mean_diff': d_mean,
        'modelo1_mejor': d_mean < 0  # d_mean < 0 significa modelo1 es mejor
    }


def comparaciones_multiples_dm(df, modelos, alpha=0.05, loss_function='none'):
    """
    CORRECCIÓN: Manejo robusto de comparaciones múltiples
    """
    resultados = []
    modelos_validos = [m for m in modelos if m in df.columns]
    
    if len(modelos_validos) < 2:
        print("⚠️ No hay suficientes modelos válidos para comparar")
        return pd.DataFrame(), alpha
    
    n_comparaciones = len(list(combinations(modelos_validos, 2)))
    if n_comparaciones == 0:
        return pd.DataFrame(), alpha
        
    # ✅ CORRECCIÓN: Bonferroni-Holm (más potente que Bonferroni simple)
    alpha_bonferroni = alpha / n_comparaciones
    
    print(f"🔍 Realizando {n_comparaciones} comparaciones con α={alpha_bonferroni:.6f}")

    for modelo1, modelo2 in combinations(modelos_validos, 2):
        try:
            # Verificar que hay datos suficientes
            data1 = df[modelo1].dropna()
            data2 = df[modelo2].dropna()
            
            if len(data1) < 10 or len(data2) < 10:
                print(f"⚠️ Datos insuficientes para {modelo1} vs {modelo2}")
                continue
            
            dm_result = diebold_mariano_test(
                data1.values,
                data2.values,
                h=1,
                alternative='two-sided',
                loss_function=loss_function
            )
            
            significativo = dm_result['p_value'] < alpha_bonferroni
            
            # ✅ CORRECCIÓN: Interpretación más precisa
            if significativo:
                if dm_result['mean_diff'] < 0:
                    ganador = modelo1
                    explicacion = f"{modelo1} es significativamente mejor (p={dm_result['p_value']:.6f})"
                else:
                    ganador = modelo2
                    explicacion = f"{modelo2} es significativamente mejor (p={dm_result['p_value']:.6f})"
            else:
                ganador = "Empate"
                explicacion = f"No hay diferencia significativa (p={dm_result['p_value']:.3f})"

            resultados.append({
                'Modelo_1': modelo1,
                'Modelo_2': modelo2,
                'DM_Statistic': dm_result['dm_statistic_corrected'],
                'p_value': dm_result['p_value'],
                'Mean_Diff': dm_result['mean_diff'],
                'Significativo': significativo,
                'Ganador': ganador,
                'Explicacion': explicacion
            })
            
        except Exception as e:
            print(f"❌ Error comparando {modelo1} vs {modelo2}: {str(e)}")
            continue

    df_resultados = pd.DataFrame(resultados)
    
    if not df_resultados.empty:
        print(f"✅ Comparaciones completadas: {len(df_resultados)}/{n_comparaciones}")
    else:
        print("❌ No se pudo realizar ninguna comparación")
    
    return df_resultados, alpha_bonferroni


def calcular_ranking_dm(df_comparaciones, modelos):
    """
    CORRECCIÓN: Ranking basado en victorias netas y porcentaje de victorias
    """
    if df_comparaciones.empty:
        print("⚠️ No hay datos de comparaciones para calcular ranking")
        return pd.DataFrame(), pd.DataFrame()
    
    # Inicializar matriz de resultados
    n = len(modelos)
    matriz_victorias = pd.DataFrame(0, index=modelos, columns=modelos, dtype=int)
    matriz_comparaciones = pd.DataFrame(0, index=modelos, columns=modelos, dtype=int)
    
    # Llenar matrices con resultados de comparaciones
    for _, row in df_comparaciones.iterrows():
        m1, m2 = row['Modelo_1'], row['Modelo_2']
        
        if m1 in modelos and m2 in modelos:
            matriz_comparaciones.loc[m1, m2] += 1
            matriz_comparaciones.loc[m2, m1] += 1
            
            if row['Significativo']:
                if row['Ganador'] == m1:
                    matriz_victorias.loc[m1, m2] += 1
                elif row['Ganador'] == m2:
                    matriz_victorias.loc[m2, m1] += 1
                # Empates no añaden victorias
    
    # ✅ CORRECCIÓN: Calcular métricas de ranking mejoradas
    ranking_data = []
    for modelo in modelos:
        victorias = matriz_victorias.loc[modelo].sum()
        comparaciones_totales = matriz_comparaciones.loc[modelo].sum()
        
        # Evitar división por cero
        if comparaciones_totales > 0:
            porcentaje_victorias = (victorias / comparaciones_totales) * 100
        else:
            porcentaje_victorias = 0
        
        # Calcular score basado en victorias netas
        derrotas = matriz_victorias.loc[:, modelo].sum()  # Victorias de otros sobre este modelo
        score_neto = victorias - derrotas
        
        ranking_data.append({
            'Modelo': modelo,
            'Victorias': int(victorias),
            'Derrotas': int(derrotas),
            'Comparaciones_Totales': int(comparaciones_totales),
            'Porcentaje_Victorias': round(porcentaje_victorias, 2),
            'Score_Neto': int(score_neto)
        })
    
    df_ranking = pd.DataFrame(ranking_data)
    
    # ✅ CORRECCIÓN: Ordenar por múltiples criterios
    df_ranking = df_ranking.sort_values(
        ['Score_Neto', 'Porcentaje_Victorias', 'Victorias'], 
        ascending=[False, False, False]
    ).reset_index(drop=True)
    
    df_ranking['Rank'] = range(1, len(df_ranking) + 1)
    
    # Reordenar columnas
    column_order = ['Rank', 'Modelo', 'Score_Neto', 'Porcentaje_Victorias', 
                   'Victorias', 'Derrotas', 'Comparaciones_Totales']
    df_ranking = df_ranking[column_order]
    
    return df_ranking, matriz_victorias

def verificar_resultados_dm(df, modelos, top_n=3):
    """
    CORRECCIÓN: Verificación más completa y robusta
    """
    print("\n" + "="*80)
    print("🔍 VERIFICACIÓN COMPLETA DE RESULTADOS DM")
    print("="*80)
    
    # 1. Estadísticas descriptivas robustas
    print("\n📊 ESTADÍSTICAS DESCRIPTIVAS (ECRPS - menor = mejor):")
    stats_df = df[modelos].describe(percentiles=[.25, .5, .75]).T
    stats_df['IQR'] = stats_df['75%'] - stats_df['25%']
    
    print(stats_df[['mean', '50%', 'std', 'IQR']].round(6))
    
    # 2. Ranking por mediana
    medianas = df[modelos].median().sort_values()
    print(f"\n🏆 RANKING POR MEDIANA:")
    for i, (modelo, mediana) in enumerate(medianas.items(), 1):
        print(f"  {i:2d}. {modelo:20s}: {mediana:.6f}")
    
    # 3. Test DM y ranking
    print(f"\n📈 REALIZANDO TEST DIEBOLD-MARIANO...")
    df_comp, alpha_bonf = comparaciones_multiples_dm(df, modelos, loss_function='none')
    
    if df_comp.empty:
        print("❌ No se pudieron realizar comparaciones DM")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    df_ranking, matriz = calcular_ranking_dm(df_comp, modelos)
    
    print(f"\n🏆 RANKING SEGÚN TEST DM (α={alpha_bonf:.6f}):")
    for _, row in df_ranking.head(top_n).iterrows():
        print(f"  {row['Rank']:2d}. {row['Modelo']:20s} "
              f"(Score: {row['Score_Neto']:3d}, Victorias: {row['Victorias']:2d}/{row['Comparaciones_Totales']:2d})")
    
    # 4. Verificar consistencia entre métodos
    print("\n🔍 VERIFICACIÓN DE CONSISTENCIA:")
    top_dm = set(df_ranking.head(top_n)['Modelo'])
    top_mediana = set(medianas.head(top_n).index)
    
    coincidencias = top_dm.intersection(top_mediana)
    if len(coincidencias) == top_n:
        print("  ✅ Consistencia PERFECTA entre DM y medianas")
    else:
        print(f"  ⚠️ Consistencia PARCIAL: {len(coincidencias)}/{top_n} coincidencias")
        if top_dm - top_mediana:
            print(f"     - Solo en Top DM: {top_dm - top_mediana}")
        if top_mediana - top_dm:
            print(f"     - Solo en Top Mediana: {top_mediana - top_dm}")
    
    # 5. Resumen de comparaciones significativas
    print(f"\n📋 RESUMEN COMPARACIONES SIGNIFICATIVAS:")
    sig_comparisons = df_comp[df_comp['Significativo']]
    if not sig_comparisons.empty:
        for _, row in sig_comparisons.iterrows():
            print(f"  ✅ {row['Explicacion']}")
    else:
        print("  ⚠️ No hay comparaciones significativas")
    
    print("\n" + "="*80)
    
    return df_ranking, matriz, df_comp

# ============================================================================
# CLASE PRINCIPAL DE ANÁLISIS
# ============================================================================

class AnalizadorBaseCompleta:

    def __init__(self, ruta_datos):
        print("\n" + "=" * 80)
        print("INICIANDO ANÁLISIS ESTADÍSTICO ROBUSTO (GENERAL + DESAGREGADO + EXCEL)")
        print("=" * 80 + "\n")

        self.df = pd.read_excel(ruta_datos)
        
        if 'proces_simulacion' in self.df.columns:
            self.df['Tipo de Modelo'] = self.df['proces_simulacion']
        else:
            raise ValueError("No se encontró 'proces_simulacion' en el Excel.")

        self.modelos = MODELOS
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)

        self._extraer_caracteristicas()
        self._preparar_escenarios()
        self.preprocessor, self.X_processed = self._preprocess_meta_features()
        self.meta_models = {} 

        print(f"✓ Datos cargados: {self.df.shape[0]} filas")
        print(f"✓ Directorio: {self.dir_salida}")
        print("\n" + "=" * 80 + "\n")

    def _extraer_caracteristicas(self):
        grupo_a = ['AR(1)', 'AR(2)', 'MA(1)', 'MA(2)', 'ARMA(1,1)', 'ARMA(2,2)', 'ARMA(2,1)']
        grupo_b = ['ARIMA(0,1,0)', 'ARIMA(1,1,0)', 'ARIMA(2,1,0)', 'ARIMA(0,1,1)', 'ARIMA(0,1,2)', 'ARIMA(1,1,1)', 'ARIMA(2,1,2)']
        grupo_c = ['SETAR-1', 'SETAR-2', 'SETAR-3', 'SETAR-4', 'SETAR-5', 'SETAR-6', 'SETAR-7']

        def clasificar_est(m):
            m = str(m).strip()
            if m in grupo_a or m in grupo_c: return 'Estacionario'
            elif m in grupo_b: return 'No Estacionario'
            return 'Desconocido'

        def clasificar_lin(m):
            m = str(m).strip()
            if m in grupo_a or m in grupo_b: return 'Lineal'
            elif m in grupo_c: return 'No Lineal'
            return 'Desconocido'

        self.df['Estacionario'] = self.df['Tipo de Modelo'].apply(clasificar_est)
        self.df['Lineal'] = self.df['Tipo de Modelo'].apply(clasificar_lin)

    def _preparar_escenarios(self):
        self.df['Escenario_Combinado'] = self.df['Estacionario'] + ' - ' + self.df['Lineal']
        mapa_nombres = {
            'Estacionario - Lineal': 'Estacionario - Lineal (ARMA)',
            'No Estacionario - Lineal': 'No Estacionario - Lineal (ARIMA)',
            'Estacionario - No Lineal': 'Estacionario - No Lineal (SETAR)'
        }
        self.df['Escenario_Combinado'] = self.df['Escenario_Combinado'].replace(mapa_nombres)
        self.escenarios_unicos = sorted(self.df['Escenario_Combinado'].unique())

    def _preprocess_meta_features(self):
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', 'passthrough', CARACTERISTICAS_NUMERICAS_META_MODELO),
                ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), CARACTERISTICAS_CATEGORICAS_META_MODELO)
            ], remainder='drop'
        )
        X = self.df[CARACTERISTICAS_META_MODELO]
        X_processed = preprocessor.fit_transform(X)
        try:
            names = list(CARACTERISTICAS_NUMERICAS_META_MODELO) + list(preprocessor.named_transformers_['cat'].get_feature_names_out())
        except:
            names = list(CARACTERISTICAS_NUMERICAS_META_MODELO) + list(preprocessor.named_transformers_['cat'].get_feature_names())
        return preprocessor, pd.DataFrame(X_processed, columns=names)

    def ejecutar_analisis_completo(self):
        print("1️⃣  Analizando Escenarios (Métricas Robustas)...")
        self._analisis_estacionariedad() 

        print("3️⃣  Analizando Modelo Generador (IQR)...")
        self._analisis_modelo_generador()

        print("4️⃣  Analizando Distribución...")
        self._analisis_distribucion()

        print("5️⃣  Analizando Varianza (Pendiente Theil-Sen)...")
        self._analisis_varianza()

        print("6️⃣  Analizando Horizonte (Pendiente Theil-Sen)...")
        self._analisis_horizonte()

        print("7️⃣  Analizando Robustez (QCD)...")
        self._analisis_robustez()

        print("8️⃣  Analizando Significancia DM y Generando Excel...")
        self._analisis_significancia()

        print("\n✅ ANÁLISIS FINALIZADO")

    # ========================================================================
    # 1. ESCENARIOS 
    # ========================================================================
    def _analisis_estacionariedad(self):
        stats_esc = []
        for modelo in self.modelos:
            for esc in self.escenarios_unicos:
                df_subset = self.df[self.df['Escenario_Combinado'] == esc]
                if not df_subset.empty:
                    stats_esc.append({
                        'Modelo': modelo, 'Escenario': esc,
                        'Mediana': df_subset[modelo].median() 
                    })
        
        df_stats = pd.DataFrame(stats_esc)
        if df_stats.empty: return

        # 1.1 Rendimiento
        fig, ax = plt.subplots(figsize=(16, 9))
        pivot_mediana = df_stats.pivot(index='Modelo', columns='Escenario', values='Mediana')
        colores = {'Estacionario - Lineal (ARMA)': '#2E7D32', 'Estacionario - No Lineal (SETAR)': '#66BB6A', 'No Estacionario - Lineal (ARIMA)': '#F57C00'}
        pivot_mediana.plot(kind='bar', ax=ax, width=0.8, color=[colores.get(c, '#333') for c in pivot_mediana.columns])
        ax.set_title('1.1 Rendimiento por Escenario (Mediana ECRPS - Robusto)', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_1_escenarios_rendimiento_mediana.png', dpi=300)
        plt.close()

        # 1.2 Relativo
        baseline = 'Estacionario - Lineal (ARMA)'
        if baseline in pivot_mediana.columns and pivot_mediana.shape[1] > 1:
            fig, ax = plt.subplots(figsize=(14, 10))
            df_rel = pivot_mediana.copy()
            for col in df_rel.columns:
                base_val = pivot_mediana[baseline]
                df_rel[col] = np.where(base_val > 1e-6, (df_rel[col] - base_val) / base_val * 100, 0)
            df_rel = df_rel.drop(columns=[baseline], errors='ignore')
            df_rel.plot(kind='barh', ax=ax, width=0.7)
            ax.set_title('1.2 Deterioro Relativo de la Mediana vs Base (%)', fontweight='bold')
            ax.axvline(0, color='k')
            plt.tight_layout()
            plt.savefig(self.dir_salida / '1_2_escenarios_cambio_relativo_robusto.png', dpi=300)
            plt.close()

    def _analisis_linealidad(self): pass

    # ========================================================================
    # 3. MODELO GENERADOR
    # ========================================================================
    def _analisis_modelo_generador(self):
        def plot_mg(df_curr, suffix):
            if df_curr.empty: return
            pivot_vals = df_curr.groupby('Tipo de Modelo')[self.modelos].median()
            tipos = df_curr['Tipo de Modelo'].unique()
            if len(tipos) < 1: return

            # 3.2 Z-Score
            fig, ax = plt.subplots(figsize=(18, 10))
            row_medians = pivot_vals.T.median(axis=1)
            row_stds = pivot_vals.T.std(axis=1)
            row_stds[row_stds == 0] = 1 
            pivot_norm = pivot_vals.T.sub(row_medians, axis=0).div(row_stds, axis=0)
            sns.heatmap(pivot_norm, annot=True, fmt='.2f', cmap='RdBu_r', center=0, ax=ax)
            ax.set_title(f'3.2 Z-Score Rendimiento Mediano ({suffix})', fontweight='bold')
            plt.tight_layout()
            plt.savefig(self.dir_salida / f'3_2_modelo_generador_zscore_{suffix}.png', dpi=300)
            plt.close()

            # 3.3 Variabilidad
            fig, ax = plt.subplots(figsize=(12, 8))
            iqr_data = []
            for t in tipos:
                data_tipo = df_curr[df_curr['Tipo de Modelo'] == t][self.modelos]
                iqrs_modelos = []
                for mod in self.modelos:
                    q75, q25 = np.percentile(data_tipo[mod].dropna(), [75 ,25])
                    iqrs_modelos.append(q75 - q25)
                iqr_data.append({'Tipo': t, 'IQR_Promedio': np.mean(iqrs_modelos)})
            if iqr_data:
                df_iqr = pd.DataFrame(iqr_data).sort_values('IQR_Promedio')
                ax.barh(df_iqr['Tipo'], df_iqr['IQR_Promedio'], color='steelblue', alpha=0.7)
                ax.set_title(f'3.3 Variabilidad Robusta (IQR Promedio) ({suffix})', fontweight='bold')
                plt.tight_layout()
                plt.savefig(self.dir_salida / f'3_3_modelo_generador_variabilidad_iqr_{suffix}.png', dpi=300)
                plt.close()

        plot_mg(self.df, "General")
        for esc in self.escenarios_unicos:
            df_esc = self.df[self.df['Escenario_Combinado'] == esc]
            clean_name = esc.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '')
            plot_mg(df_esc, clean_name)

    # ========================================================================
    # 4. DISTRIBUCIÓN
    # ========================================================================
    def _analisis_distribucion(self):
        def plot_dist(df_curr, suffix):
            if df_curr.empty: return
            pivot_med = df_curr.groupby('Distribución')[self.modelos].median()
            pivot_iqr = pd.DataFrame(index=df_curr['Distribución'].unique(), columns=self.modelos)
            for dist in pivot_iqr.index:
                for mod in self.modelos:
                    subset = df_curr[df_curr['Distribución'] == dist][mod].dropna()
                    if not subset.empty:
                        q75, q25 = np.percentile(subset, [75, 25])
                        pivot_iqr.loc[dist, mod] = q75 - q25
            pivot_iqr = pivot_iqr.astype(float)
            if pivot_med.empty: return

            fig, ax = plt.subplots(figsize=(14, 10))
            sns.heatmap(pivot_med.T, annot=True, fmt='.3f', cmap='RdYlGn_r', ax=ax)
            ax.set_title(f'4.1 Mediana ECRPS por Distribución ({suffix})', fontweight='bold')
            plt.tight_layout()
            plt.savefig(self.dir_salida / f'4_1_distribucion_heatmap_rendimiento_{suffix}.png', dpi=300)
            plt.close()

            fig, ax = plt.subplots(figsize=(14, 10))
            sns.heatmap(pivot_iqr.T, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax)
            ax.set_title(f'4.2 Variabilidad Robusta (IQR) por Distribución ({suffix})', fontweight='bold')
            plt.tight_layout()
            plt.savefig(self.dir_salida / f'4_2_distribucion_heatmap_variabilidad_{suffix}.png', dpi=300)
            plt.close()

        plot_dist(self.df, "General")
        for esc in self.escenarios_unicos:
            df_esc = self.df[self.df['Escenario_Combinado'] == esc]
            clean_name = esc.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '')
            plot_dist(df_esc, clean_name)

    # ========================================================================
    # 5. VARIANZA
    # ========================================================================
    def _analisis_varianza(self):
        varianzas = sorted(self.df['Varianza error'].unique())
        
        def plot_varianza(df_curr, suffix, title_extra):
            if df_curr.empty: return
            
            # 5.1 Tendencias
            fig, ax = plt.subplots(figsize=(14, 8))
            for modelo in self.modelos:
                medianas = []
                vars_presentes = []
                for v in varianzas:
                    subset = df_curr[df_curr['Varianza error'] == v][modelo]
                    if not subset.empty:
                        medianas.append(subset.median())
                        vars_presentes.append(v)
                if medianas:
                    ax.plot(vars_presentes, medianas, marker='o', label=modelo, color=COLORES_MODELOS[modelo])
            ax.set_title(f'5.1 Deterioro Varianza (Mediana) - {title_extra}', fontweight='bold')
            ax.legend(bbox_to_anchor=(1.01, 1))
            plt.tight_layout()
            plt.savefig(self.dir_salida / f'5_1_varianza_tendencias_{suffix}.png', dpi=300)
            plt.close()

            # 5.2 Tasas
            fig, ax = plt.subplots(figsize=(12, 8))
            tasas_theil = {}
            for modelo in self.modelos:
                datos_mod = df_curr[['Varianza error', modelo]].dropna()
                if len(datos_mod) > 2:
                    x = datos_mod['Varianza error'].values
                    y = datos_mod[modelo].values
                    res = stats.theilslopes(y, x, alpha=0.95)
                    tasas_theil[modelo] = res[0]
            if tasas_theil:
                tasas_theil = dict(sorted(tasas_theil.items(), key=lambda x: x[1]))
                vals = list(tasas_theil.values())
                median_slope = np.median(vals)
                colors = ['green' if v < median_slope else 'red' for v in vals]
                ax.barh(list(tasas_theil.keys()), vals, color=colors, alpha=0.7, edgecolor='k')
                ax.set_title(f'5.2 Sensibilidad Ruido (Pendiente Theil-Sen) - {title_extra}', fontweight='bold')
                plt.tight_layout()
                plt.savefig(self.dir_salida / f'5_2_varianza_tasa_{suffix}.png', dpi=300)
                plt.close()

        plot_varianza(self.df, "General", "Todos los datos")
        for esc in self.escenarios_unicos:
            df_esc = self.df[self.df['Escenario_Combinado'] == esc]
            clean_name = esc.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '')
            plot_varianza(df_esc, clean_name, esc)

    # ========================================================================
    # 6. HORIZONTE
    # ========================================================================
    def _analisis_horizonte(self):
        def plot_horizonte(df_curr, suffix, title_extra):
            if df_curr.empty: return
            
            # 6.1 Evolución
            fig, ax = plt.subplots(figsize=(14, 8))
            for modelo in self.modelos:
                medianas = df_curr.groupby('Paso')[modelo].median()
                ax.plot(medianas.index, medianas.values, marker='o', label=modelo, color=COLORES_MODELOS[modelo], linewidth=2)
            ax.set_title(f'6.1 Evolución por Horizonte (Mediana) ({title_extra})', fontweight='bold')
            ax.legend(bbox_to_anchor=(1.01, 1))
            plt.tight_layout()
            plt.savefig(self.dir_salida / f'6_1_horizonte_evolucion_{suffix}.png', dpi=300)
            plt.close()

            # 6.2 Deterioro
            fig, ax = plt.subplots(figsize=(12, 8))
            deterioros = {}
            for modelo in self.modelos:
                datos_mod = df_curr[['Paso', modelo]].dropna()
                if len(datos_mod) > 5: 
                    x = datos_mod['Paso'].values
                    y = datos_mod[modelo].values
                    res = stats.theilslopes(y, x, alpha=0.95)
                    deterioros[modelo] = res[0]
            if deterioros:
                deterioros = dict(sorted(deterioros.items(), key=lambda x: x[1]))
                median_val = np.median(list(deterioros.values()))
                colors = ['green' if v < median_val else 'red' for v in deterioros.values()]
                ax.barh(list(deterioros.keys()), list(deterioros.values()), color=colors, alpha=0.7, edgecolor='k')
                ax.set_title(f'6.2 Velocidad de Deterioro (Pendiente Theil-Sen) ({title_extra})', fontweight='bold')
                plt.tight_layout()
                plt.savefig(self.dir_salida / f'6_2_horizonte_tasa_{suffix}.png', dpi=300)
                plt.close()

        plot_horizonte(self.df, "General", "Todos los datos")
        for esc in self.escenarios_unicos:
            df_esc = self.df[self.df['Escenario_Combinado'] == esc]
            clean_name = esc.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '')
            plot_horizonte(df_esc, clean_name, esc)

    # ========================================================================
    # 7. ROBUSTEZ
    # ========================================================================
    def _analisis_robustez(self):
        metricas = []
        for modelo in self.modelos:
            datos = self.df[modelo].dropna()
            if len(datos) > 0:
                q75, q25 = np.percentile(datos, [75, 25])
                if (q75 + q25) > 0:
                    qcd = (q75 - q25) / (q75 + q25)
                    metricas.append({'Modelo': modelo, 'QCD': qcd})
        df_r = pd.DataFrame(metricas).sort_values('QCD')
        
        fig, ax = plt.subplots(figsize=(12, 8))
        colors = plt.cm.RdYlGn(np.linspace(0.8, 0.2, len(df_r)))
        bars = ax.barh(df_r['Modelo'], df_r['QCD'], color=colors, edgecolor='k')
        for i, (bar, val) in enumerate(zip(bars, df_r['QCD'])):
            ax.text(val, i, f'{val:.3f}', va='center', fontweight='bold')
        ax.set_title('7.2 Robustez (Coeficiente de Dispersión por Cuartiles - QCD)', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '7_2_robustez_general_qcd.png', dpi=300)
        plt.close()

    # ========================================================================
    # 8. SIGNIFICANCIA (HEATMAP + EXCEL)
    # ========================================================================
    def _analisis_significancia(self):
        """Versión corregida del análisis de significancia"""
        
        nombre_excel = self.dir_salida / "Tabla_Victorias_DM_CORREGIDO.xlsx"
        
        sheet_mapping = {
            'General': 'General',
            'Estacionario - Lineal (ARMA)': 'Est_Lin_ARMA',
            'No Estacionario - Lineal (ARIMA)': 'NoEst_Lin_ARIMA',
            'Estacionario - No Lineal (SETAR)': 'Est_NoLin_SETAR'
        }

        with pd.ExcelWriter(nombre_excel, engine='openpyxl') as writer:
            
            def procesar_dm(df_curr, suffix, sheet_name_full):
                if df_curr.empty or len(df_curr) < 10: 
                    print(f"   ⚠️ Saltando DM para {suffix} (Datos insuficientes)")
                    return

                print(f"\n   {'='*60}")
                print(f"   Procesando: {suffix}")
                print(f"   {'='*60}")
                
                # ✅ USAR FUNCIÓN DE VERIFICACIÓN
                df_ranking, matriz, df_comp = verificar_resultados_dm(
                    df_curr, self.modelos, top_n=3
                )
                
                if df_comp.empty: 
                    print(f"   ⚠️ No se pudieron realizar comparaciones para {suffix}")
                    return
                
                # Guardar en Excel
                safe_sheet_name = sheet_mapping.get(sheet_name_full, suffix[:30])
                df_ranking.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                
                # Matriz de superioridad
                fig, ax = plt.subplots(figsize=(12, 10))
                sns.heatmap(matriz, annot=True, fmt='.0f', cmap='RdYlGn', center=0, ax=ax,
                            cbar_kws={'label': 'Superioridad (1=Gana Fila, -1=Gana Col)'})
                ax.set_title(f'Matriz Superioridad DM ({suffix})', fontweight='bold')
                plt.tight_layout()
                clean_filename = suffix.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '')
                plt.savefig(self.dir_salida / f'8_2_significancia_matriz_{clean_filename}.png', dpi=300)
                plt.close()

            # Procesar general y desagregado
            procesar_dm(self.df, "General", "General")
            
            for esc in self.escenarios_unicos:
                df_esc = self.df[self.df['Escenario_Combinado'] == esc]
                procesar_dm(df_esc, esc, esc)
        
        print(f"\n   📊 Excel corregido generado: {nombre_excel}")

# ============================================================================
# MAIN
# ============================================================================
def main():
    try:
        analizador = AnalizadorBaseCompleta(RUTA_DATOS)
        analizador.ejecutar_analisis_completo()
    except FileNotFoundError:
        print(f"\n❌ ERROR: No archivo {RUTA_DATOS}")
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


INICIANDO ANÁLISIS ESTADÍSTICO ROBUSTO (GENERAL + DESAGREGADO + EXCEL)

✓ Datos cargados: 5040 filas
✓ Directorio: resultados_generales_robustos


1️⃣  Analizando Escenarios (Métricas Robustas)...
3️⃣  Analizando Modelo Generador (IQR)...
4️⃣  Analizando Distribución...
5️⃣  Analizando Varianza (Pendiente Theil-Sen)...
6️⃣  Analizando Horizonte (Pendiente Theil-Sen)...
7️⃣  Analizando Robustez (QCD)...
8️⃣  Analizando Significancia DM y Generando Excel...

   Procesando: General

🔍 VERIFICACIÓN COMPLETA DE RESULTADOS DM

📊 ESTADÍSTICAS DESCRIPTIVAS (ECRPS - menor = mejor):
                         mean       50%       std       IQR
AREPD                3.809377  0.949502  8.880501  2.568691
AV-MCPS              1.480903  0.593915  3.222711  1.010705
Block Bootstrapping  4.177433  0.907520  9.686845  2.912166
DeepAR               1.449646  0.498318  4.592738  0.730393
EnCQR-LSTM           2.569865  1.161221  4.526081  1.953390
LSPM                 0.852910  0.513936  0.997760  0.755655

## Ranking

In [11]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Crear carpeta de resultados si no existe
output_dir = Path("./resultados_rankings_comparacion")
output_dir.mkdir(parents=True, exist_ok=True)

# Configuración
archivo_excel = "./Base_140_3_escenarios.xlsx"
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Mapeo de nombres de escenarios
ESCENARIOS_MAP = {
    'Lineal - estacionario': 'Lineal - estacionario (ARMA)',
    'Lineal - NO estacionario': 'Lineal - NO estacionario (ARIMA)',
    'NO lineal - estacionario': 'NO lineal - estacionario (SETAR)'
}

# Cargar datos
print("Cargando datos...")
df = pd.read_excel(archivo_excel)

# Verificar columnas
columnas_faltantes = [modelo for modelo in MODELOS if modelo not in df.columns]
if columnas_faltantes:
    print(f"Advertencia: Las siguientes columnas no se encontraron: {columnas_faltantes}")
    MODELOS = [m for m in MODELOS if m in df.columns]

if 'ESCENARIO' not in df.columns:
    raise ValueError("La columna 'ESCENARIO' no se encontró en el archivo Excel")

print(f"Modelos a comparar: {MODELOS}")
print(f"Escenarios encontrados: {df['ESCENARIO'].unique()}")

# ============================================================================
# FUNCIONES PARA ANÁLISIS CON DIEBOLD-MARIANO (CON SIGNIFICANCIA)
# ============================================================================

def diebold_mariano_test(errors1, errors2):
    """Test de Diebold-Mariano para comparar dos series de errores."""
    d = errors1**2 - errors2**2
    d = d.dropna()
    
    if len(d) < 2:
        return np.nan, np.nan
    
    d_mean = d.mean()
    n = len(d)
    d_var = d.var() / n
    
    if d_var <= 0:
        return np.nan, np.nan
    
    dm_stat = d_mean / np.sqrt(d_var)
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    
    return dm_stat, p_value

def procesar_escenario_DM(df_filtrado, nombre_escenario, MODELOS):
    """Procesa un escenario con test DM y corrección de Bonferroni."""
    n_modelos = len(MODELOS)
    matriz_resultados = np.zeros((n_modelos, n_modelos))
    
    n_comparaciones = n_modelos * (n_modelos - 1) / 2
    alpha_corregido = 0.05 / n_comparaciones
    
    print(f"\n{'='*80}")
    print(f"DM - Procesando: {nombre_escenario}")
    print(f"Observaciones: {len(df_filtrado)}")
    print(f"Alpha (Bonferroni): {alpha_corregido:.6f}")
    
    # Realizar comparaciones
    for i, modelo1 in enumerate(MODELOS):
        for j, modelo2 in enumerate(MODELOS):
            if i == j:
                matriz_resultados[i, j] = 0
            elif i < j:
                errors1 = df_filtrado[modelo1]
                errors2 = df_filtrado[modelo2]
                
                dm_stat, p_value = diebold_mariano_test(errors1, errors2)
                
                if np.isnan(p_value):
                    matriz_resultados[i, j] = 0
                    matriz_resultados[j, i] = 0
                elif p_value < alpha_corregido:
                    mean1 = abs(errors1).mean()
                    mean2 = abs(errors2).mean()
                    
                    if mean1 < mean2:
                        matriz_resultados[i, j] = 1
                        matriz_resultados[j, i] = -1
                    else:
                        matriz_resultados[i, j] = -1
                        matriz_resultados[j, i] = 1
                else:
                    matriz_resultados[i, j] = 0
                    matriz_resultados[j, i] = 0
    
    # Calcular ranking
    resultados_ranking = []
    for i, modelo in enumerate(MODELOS):
        victorias = np.sum(matriz_resultados[i, :] == 1)
        derrotas = np.sum(matriz_resultados[i, :] == -1)
        empates = np.sum(matriz_resultados[i, :] == 0) - 1  # -1 para excluir diagonal
        score_neto = victorias - derrotas
        comparaciones_totales = n_modelos - 1
        porcentaje_victorias = (victorias / comparaciones_totales) * 100
        
        resultados_ranking.append({
            'Modelo': modelo,
            'Victorias': int(victorias),
            'Empates': int(empates),
            'Derrotas': int(derrotas),
            'Score': int(score_neto),
            '% Victorias': round(porcentaje_victorias, 2)
        })
    
    df_ranking = pd.DataFrame(resultados_ranking)
    df_ranking = df_ranking.sort_values('Score', ascending=False).reset_index(drop=True)
    df_ranking.insert(0, 'Rank', range(1, len(df_ranking) + 1))
    
    return df_ranking

# ============================================================================
# FUNCIONES PARA ANÁLISIS POR MEDIA SIMPLE (SIN SIGNIFICANCIA)
# ============================================================================

def procesar_escenario_MEDIA(df_filtrado, nombre_escenario, MODELOS):
    """Procesa un escenario comparando directamente las medias sin test de significancia."""
    n_modelos = len(MODELOS)
    matriz_resultados = np.zeros((n_modelos, n_modelos))
    
    print(f"\n{'='*80}")
    print(f"MEDIA - Procesando: {nombre_escenario}")
    print(f"Observaciones: {len(df_filtrado)}")
    
    # Calcular medias
    medias = {}
    for modelo in MODELOS:
        medias[modelo] = abs(df_filtrado[modelo]).mean()
    
    # Comparar todas las parejas por media
    for i, modelo1 in enumerate(MODELOS):
        for j, modelo2 in enumerate(MODELOS):
            if i == j:
                matriz_resultados[i, j] = 0
            else:
                if medias[modelo1] < medias[modelo2]:
                    matriz_resultados[i, j] = 1
                elif medias[modelo1] > medias[modelo2]:
                    matriz_resultados[i, j] = -1
                else:
                    matriz_resultados[i, j] = 0
    
    # Calcular ranking
    resultados_ranking = []
    for i, modelo in enumerate(MODELOS):
        victorias = np.sum(matriz_resultados[i, :] == 1)
        derrotas = np.sum(matriz_resultados[i, :] == -1)
        empates = np.sum(matriz_resultados[i, :] == 0) - 1  # -1 para excluir diagonal
        score_neto = victorias - derrotas
        comparaciones_totales = n_modelos - 1
        porcentaje_victorias = (victorias / comparaciones_totales) * 100
        
        resultados_ranking.append({
            'Modelo': modelo,
            'Victorias': int(victorias),
            'Empates': int(empates),
            'Derrotas': int(derrotas),
            'Score': int(score_neto),
            '% Victorias': round(porcentaje_victorias, 2)
        })
    
    df_ranking = pd.DataFrame(resultados_ranking)
    df_ranking = df_ranking.sort_values('Score', ascending=False).reset_index(drop=True)
    df_ranking.insert(0, 'Rank', range(1, len(df_ranking) + 1))
    
    return df_ranking

# ============================================================================
# PROCESAR TODOS LOS ESCENARIOS
# ============================================================================

resultados_DM = {}
resultados_MEDIA = {}

# 1. Análisis General (todos los escenarios)
print("\n" + "="*80)
print("PROCESANDO ANÁLISIS GENERAL (TODOS LOS ESCENARIOS)")
print("="*80)

resultados_DM['General'] = procesar_escenario_DM(df, "General", MODELOS)
resultados_MEDIA['General'] = procesar_escenario_MEDIA(df, "General", MODELOS)

# 2. Análisis por escenario específico
for escenario_original, escenario_nombre in ESCENARIOS_MAP.items():
    df_filtrado = df[df['ESCENARIO'] == escenario_original].copy()
    
    if len(df_filtrado) > 0:
        resultados_DM[escenario_nombre] = procesar_escenario_DM(df_filtrado, escenario_nombre, MODELOS)
        resultados_MEDIA[escenario_nombre] = procesar_escenario_MEDIA(df_filtrado, escenario_nombre, MODELOS)
    else:
        print(f"\nAdvertencia: No se encontraron datos para '{escenario_original}'")

# ============================================================================
# GUARDAR RESULTADOS EN EXCEL
# ============================================================================

print("\n" + "="*80)
print("GUARDANDO RESULTADOS EN EXCEL")
print("="*80)

# Excel con método Diebold-Mariano
archivo_ranking_DM = output_dir / "ranking_modelos_DM.xlsx"
with pd.ExcelWriter(archivo_ranking_DM, engine='openpyxl') as writer:
    resultados_DM['General'].to_excel(writer, sheet_name='General', index=False)
    
    for escenario_nombre in ESCENARIOS_MAP.values():
        if escenario_nombre in resultados_DM:
            if 'ARMA' in escenario_nombre:
                sheet_name = 'ARMA'
            elif 'ARIMA' in escenario_nombre:
                sheet_name = 'ARIMA'
            elif 'SETAR' in escenario_nombre:
                sheet_name = 'SETAR'
            
            resultados_DM[escenario_nombre].to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Rankings DM guardados en: {archivo_ranking_DM}")

# Excel con método de Media Simple
archivo_ranking_MEDIA = output_dir / "ranking_modelos_MEDIA.xlsx"
with pd.ExcelWriter(archivo_ranking_MEDIA, engine='openpyxl') as writer:
    resultados_MEDIA['General'].to_excel(writer, sheet_name='General', index=False)
    
    for escenario_nombre in ESCENARIOS_MAP.values():
        if escenario_nombre in resultados_MEDIA:
            if 'ARMA' in escenario_nombre:
                sheet_name = 'ARMA'
            elif 'ARIMA' in escenario_nombre:
                sheet_name = 'ARIMA'
            elif 'SETAR' in escenario_nombre:
                sheet_name = 'SETAR'
            
            resultados_MEDIA[escenario_nombre].to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Rankings MEDIA guardados en: {archivo_ranking_MEDIA}")

# ============================================================================
# FUNCIONES PARA CREAR MATRICES DE COMPARACIÓN
# ============================================================================

def crear_matriz_comparacion_DM(df_filtrado, MODELOS):
    """Crea matriz de comparaciones con test DM."""
    n_modelos = len(MODELOS)
    matriz_resultados = np.zeros((n_modelos, n_modelos))
    
    n_comparaciones = n_modelos * (n_modelos - 1) / 2
    alpha_corregido = 0.05 / n_comparaciones
    
    for i, modelo1 in enumerate(MODELOS):
        for j, modelo2 in enumerate(MODELOS):
            if i == j:
                matriz_resultados[i, j] = 0
            elif i < j:
                errors1 = df_filtrado[modelo1]
                errors2 = df_filtrado[modelo2]
                
                dm_stat, p_value = diebold_mariano_test(errors1, errors2)
                
                if np.isnan(p_value):
                    matriz_resultados[i, j] = 0
                    matriz_resultados[j, i] = 0
                elif p_value < alpha_corregido:
                    mean1 = abs(errors1).mean()
                    mean2 = abs(errors2).mean()
                    
                    if mean1 < mean2:
                        matriz_resultados[i, j] = 1
                        matriz_resultados[j, i] = -1
                    else:
                        matriz_resultados[i, j] = -1
                        matriz_resultados[j, i] = 1
                else:
                    matriz_resultados[i, j] = 0
                    matriz_resultados[j, i] = 0
    
    return matriz_resultados

def crear_matriz_comparacion_MEDIA(df_filtrado, MODELOS):
    """Crea matriz de comparaciones por media simple."""
    n_modelos = len(MODELOS)
    matriz_resultados = np.zeros((n_modelos, n_modelos))
    
    medias = {}
    for modelo in MODELOS:
        medias[modelo] = abs(df_filtrado[modelo]).mean()
    
    for i, modelo1 in enumerate(MODELOS):
        for j, modelo2 in enumerate(MODELOS):
            if i == j:
                matriz_resultados[i, j] = 0
            else:
                if medias[modelo1] < medias[modelo2]:
                    matriz_resultados[i, j] = 1
                elif medias[modelo1] > medias[modelo2]:
                    matriz_resultados[i, j] = -1
                else:
                    matriz_resultados[i, j] = 0
    
    return matriz_resultados

# ============================================================================
# VISUALIZACIONES - HEATMAPS INDIVIDUALES (8 GRÁFICOS)
# ============================================================================

print("\n" + "="*80)
print("GENERANDO VISUALIZACIONES - HEATMAPS INDIVIDUALES")
print("="*80)

escenarios_orden = ['General'] + list(ESCENARIOS_MAP.values())
cmap = sns.diverging_palette(10, 130, as_cmap=True)

# Generar 4 heatmaps para DM
for escenario_nombre in escenarios_orden:
    if escenario_nombre in resultados_DM:
        # Obtener datos según escenario
        if escenario_nombre == 'General':
            df_filtrado = df
        else:
            escenario_original = [k for k, v in ESCENARIOS_MAP.items() if v == escenario_nombre][0]
            df_filtrado = df[df['ESCENARIO'] == escenario_original].copy()
        
        matriz = crear_matriz_comparacion_DM(df_filtrado, MODELOS)
        mask = np.eye(len(MODELOS), dtype=bool)
        
        fig, ax = plt.subplots(figsize=(12, 10))
        
        sns.heatmap(matriz,
                    annot=True,
                    fmt='.0f',
                    cmap=cmap,
                    center=0,
                    vmin=-1,
                    vmax=1,
                    xticklabels=MODELOS,
                    yticklabels=MODELOS,
                    cbar_kws={'label': 'Resultado (1=Fila Gana, 0=Empate, -1=Fila Pierde)'},
                    linewidths=0.5,
                    linecolor='gray',
                    mask=mask,
                    ax=ax)
        
        plt.title(f'Matriz de Comparaciones: {escenario_nombre}\n' +
                  f'Método: Diebold-Mariano con Corrección de Bonferroni (n={len(df_filtrado)})\n' +
                  'Verde=Fila Gana | Amarillo=Sin diferencia | Rojo=Fila Pierde',
                  fontsize=12, fontweight='bold', pad=20)
        plt.xlabel('Modelo (Columna)', fontsize=10)
        plt.ylabel('Modelo (Fila)', fontsize=10)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        
        nombre_archivo = escenario_nombre.replace(' ', '_').replace('(', '').replace(')', '')
        archivo_matriz = output_dir / f"matriz_DM_{nombre_archivo}.png"
        plt.savefig(archivo_matriz, dpi=300, bbox_inches='tight')
        print(f"Heatmap DM guardado: {archivo_matriz}")
        plt.close()

# Generar 4 heatmaps para MEDIA
for escenario_nombre in escenarios_orden:
    if escenario_nombre in resultados_MEDIA:
        # Obtener datos según escenario
        if escenario_nombre == 'General':
            df_filtrado = df
        else:
            escenario_original = [k for k, v in ESCENARIOS_MAP.items() if v == escenario_nombre][0]
            df_filtrado = df[df['ESCENARIO'] == escenario_original].copy()
        
        matriz = crear_matriz_comparacion_MEDIA(df_filtrado, MODELOS)
        mask = np.eye(len(MODELOS), dtype=bool)
        
        fig, ax = plt.subplots(figsize=(12, 10))
        
        sns.heatmap(matriz,
                    annot=True,
                    fmt='.0f',
                    cmap=cmap,
                    center=0,
                    vmin=-1,
                    vmax=1,
                    xticklabels=MODELOS,
                    yticklabels=MODELOS,
                    cbar_kws={'label': 'Resultado (1=Fila Gana, 0=Empate, -1=Fila Pierde)'},
                    linewidths=0.5,
                    linecolor='gray',
                    mask=mask,
                    ax=ax)
        
        plt.title(f'Matriz de Comparaciones: {escenario_nombre}\n' +
                  f'Método: Comparación Directa por Media (n={len(df_filtrado)})\n' +
                  'Verde=Fila Gana | Amarillo=Sin diferencia | Rojo=Fila Pierde',
                  fontsize=12, fontweight='bold', pad=20)
        plt.xlabel('Modelo (Columna)', fontsize=10)
        plt.ylabel('Modelo (Fila)', fontsize=10)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        
        nombre_archivo = escenario_nombre.replace(' ', '_').replace('(', '').replace(')', '')
        archivo_matriz = output_dir / f"matriz_MEDIA_{nombre_archivo}.png"
        plt.savefig(archivo_matriz, dpi=300, bbox_inches='tight')
        print(f"Heatmap MEDIA guardado: {archivo_matriz}")
        plt.close()

# ============================================================================
# RESUMEN FINAL
# ============================================================================

print("\n" + "="*80)
print("RESUMEN FINAL - COMPARACIÓN DE MÉTODOS")
print("="*80)

for escenario_nombre in escenarios_orden:
    if escenario_nombre in resultados_DM:
        print(f"\n{escenario_nombre}:")
        
        print("  Top 3 - Diebold-Mariano:")
        df_dm = resultados_DM[escenario_nombre]
        for idx, row in df_dm.head(3).iterrows():
            print(f"    {int(row['Rank'])}. {row['Modelo']}: Score={int(row['Score'])}, " +
                  f"V={int(row['Victorias'])}, E={int(row['Empates'])}, D={int(row['Derrotas'])} ({row['% Victorias']:.1f}%)")
        
        print("  Top 3 - Media Simple:")
        df_media = resultados_MEDIA[escenario_nombre]
        for idx, row in df_media.head(3).iterrows():
            print(f"    {int(row['Rank'])}. {row['Modelo']}: Score={int(row['Score'])}, " +
                  f"V={int(row['Victorias'])}, E={int(row['Empates'])}, D={int(row['Derrotas'])} ({row['% Victorias']:.1f}%)")

print("\n" + "="*80)
print("ARCHIVOS GENERADOS")
print("="*80)
print(f"📁 Directorio: {output_dir}")
print(f"📊 Excel DM: ranking_modelos_DM.xlsx (4 hojas)")
print(f"📊 Excel MEDIA: ranking_modelos_MEDIA.xlsx (4 hojas)")
print(f"🖼️  Heatmaps DM individuales: matriz_DM_[escenario].png (4 archivos)")
print(f"🖼️  Heatmaps MEDIA individuales: matriz_MEDIA_[escenario].png (4 archivos)")
print(f"   Total: 8 heatmaps individuales")
print("="*80)

Cargando datos...
Modelos a comparar: ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
Escenarios encontrados: ['Lineal - estacionario' 'Lineal - NO estacionario'
 'NO lineal - estacionario']

PROCESANDO ANÁLISIS GENERAL (TODOS LOS ESCENARIOS)

DM - Procesando: General
Observaciones: 5040
Alpha (Bonferroni): 0.001389

MEDIA - Procesando: General
Observaciones: 5040

DM - Procesando: Lineal - estacionario (ARMA)
Observaciones: 1680
Alpha (Bonferroni): 0.001389

MEDIA - Procesando: Lineal - estacionario (ARMA)
Observaciones: 1680

DM - Procesando: Lineal - NO estacionario (ARIMA)
Observaciones: 1680
Alpha (Bonferroni): 0.001389

MEDIA - Procesando: Lineal - NO estacionario (ARIMA)
Observaciones: 1680

DM - Procesando: NO lineal - estacionario (SETAR)
Observaciones: 1680
Alpha (Bonferroni): 0.001389

MEDIA - Procesando: NO lineal - estacionario (SETAR)
Observaciones: 1680

GUARDANDO RESULTADOS EN EXCEL
Rankings DM guardados en

# Analisis Diferenciado

## Pre-procesamiento

In [12]:
import pandas as pd
import numpy as np

# Leer los tres archivos
arima_df = pd.read_excel("./datos/resultados_140_ARIMA_FINAL.xlsx")
arima_Diff_df = pd.read_excel("./datos/resultados_140_ARIMA_CON_DIFERENCIACION.xlsx")

# Filtrar los que no tienen "Promedio" en la columna "Paso"
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
arima_Diff_df = arima_Diff_df[arima_Diff_df['Paso'] != 'Promedio']

# Lista de modelos (columnas a promediar)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario (de la columna del modelo)
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    arima_Diff_promedio = arima_Diff_df[modelo].mean() if modelo in arima_Diff_df.columns else np.nan
    
    fila['ARIMA'] = arima_promedio
    fila['ARIMA_Diff'] = arima_Diff_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARIMA': arima_promedio,
        'ARIMA_Diff': arima_Diff_promedio
    }
    
    # Filtrar NaN si existen
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)

# Redondear valores para mejor visualización
columnas_numericas = ['ARIMA', 'ARIMA_Diff']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")

# Guardar tabla comparativa en Excel
tabla_comparativa.to_excel("Tabla_Comparativa_Modelos_Diff.xlsx", index=False)
print("Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_Diff.xlsx'")

# Agregar columna ESCENARIO a cada DataFrame antes de concatenar
arima_df['ESCENARIO'] = 'Sin diferenciación'
arima_Diff_df['ESCENARIO'] = 'Diferenciado'

# Concatenar los tres dataframes
base_consolidada = pd.concat([arima_df, arima_Diff_df], ignore_index=True)
# Guardar en un archivo Excel
base_consolidada.to_excel("Base_140_diff_escenarios.xlsx", index=False)

print("\nArchivo 'Base_140_diff_escenarios.xlsx' creado exitosamente!")
print(f"\nTotal de filas: {len(base_consolidada)}")
print(f"- ARIMA: {len(arima_df)} filas")
print(f"- ARIMA_Diff: {len(arima_Diff_df)} filas")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARIMA  ARIMA_Diff Mejor_Escenario
              AREPD  9.7604      0.7485      ARIMA_Diff
            AV-MCPS  3.0618      0.6493      ARIMA_Diff
Block Bootstrapping 10.9690      0.6844      ARIMA_Diff
             DeepAR  3.1462      0.5704      ARIMA_Diff
         EnCQR-LSTM  5.8306      0.8656      ARIMA_Diff
               LSPM  1.1140      0.6481      ARIMA_Diff
              LSPMW  3.5094      0.8032      ARIMA_Diff
               MCPS  2.8994      0.6581      ARIMA_Diff
    Sieve Bootstrap  0.5479      0.5454      ARIMA_Diff

Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_Diff.xlsx'

Archivo 'Base_140_diff_escenarios.xlsx' creado exitosamente!

Total de filas: 3360
- ARIMA: 1680 filas
- ARIMA_Diff: 1680 filas


## Analisis general

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings
from matplotlib.patches import Patch

warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./Base_140_diff_escenarios.xlsx"
DIR_SALIDA = "./resultados_escenarios_comparativos"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Colores fijos para barras comparativas
COLOR_SIN_DIFF = '#7f7f7f'  # Gris
COLOR_CON_DIFF = '#1f77b4'  # Azul

# Paleta UNIFICADA para todos los Heatmaps
# RdYlGn_r: Rojo (Valores altos/malos) -> Verde (Valores bajos/buenos)
CMAP_HEATMAP = 'RdYlGn_r' 

# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================

def diebold_mariano_test(errores1, errores2, h=1, alternative='two-sided', loss_function='none'):
    """Test DM Robusto (HAC)"""
    e1 = np.asarray(errores1)
    e2 = np.asarray(errores2)
    n = len(e1)
    
    if loss_function == 'none':
        d = e1 - e2
    elif loss_function == 'squared':
        d = e1**2 - e2**2
    else: # absolute
        d = np.abs(e1) - np.abs(e2)
        
    d_mean = np.mean(d)
    
    if h == 1:
        var_d = np.var(d, ddof=1) / n
    else:
        gamma_0 = np.var(d, ddof=1)
        gamma_sum = 0
        max_lags = min(h-1, n-1)
        for k in range(1, max_lags + 1):
            if k < n:
                gamma_k = np.cov(d[:-k], d[k:], ddof=1)[0,1] if len(d) > k else 0
                gamma_sum += (1 - k/(max_lags+1)) * gamma_k
        var_d = (gamma_0 + 2 * gamma_sum) / n
    
    hlnc = np.sqrt((n + 1 - 2 * h + h * (h - 1) / n) / n) if h > 1 else 1.0
    
    dm_stat = (d_mean / np.sqrt(var_d)) * hlnc if var_d > 0 else 0
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    
    return {'p_value': p_value, 'mean_diff': d_mean, 'dm_statistic': dm_stat}

def comparar_modelo_entre_escenarios(df, modelo, esc_base, esc_diff, alpha=0.05):
    data_base = df[df['ESCENARIO'] == esc_base][modelo].dropna()
    data_diff = df[df['ESCENARIO'] == esc_diff][modelo].dropna()
    
    if len(data_base) < 10 or len(data_diff) < 10:
        return {'Modelo': modelo, 'Conclusión': 'Datos insuficientes', 'p_value': np.nan}
    
    med_base = data_base.median()
    med_diff = data_diff.median()
    mejora_pct = ((med_base - med_diff) / med_base) * 100 if med_base != 0 else 0
    
    try:
        res = diebold_mariano_test(data_base.values, data_diff.values, h=1, loss_function='none')
        sig = res['p_value'] < alpha
        
        if sig:
            conclusion = 'Diferenciación mejora' if res['mean_diff'] > 0 else 'Sin diferenciación es mejor'
        else:
            conclusion = 'Sin diferencia significativa'
            
        return {
            'Modelo': modelo,
            'ECRPS_Sin_Dif': round(med_base, 3),
            'ECRPS_Con_Dif': round(med_diff, 3),
            'Mejora_%': round(mejora_pct, 2),
            'dm_statistic': round(res['dm_statistic'], 4),
            'p_value': res['p_value'],
            'Significativo': 'Sí' if sig else 'No',
            'Conclusión': conclusion
        }
    except:
        return {'Modelo': modelo, 'Conclusión': 'Error'}

# ============================================================================
# CLASE PRINCIPAL
# ============================================================================

class AnalizadorBaseCompleta:

    def __init__(self, ruta_datos):
        print("\n" + "=" * 80)
        print("INICIANDO ANÁLISIS COMPARATIVO (AJUSTADO - BARRAS HORIZONTALES)")
        print("=" * 80 + "\n")

        self.df = pd.read_excel(ruta_datos)
        
        # Limpieza básica
        self.df['ESCENARIO'] = self.df['ESCENARIO'].astype(str).str.strip()
        self.escenarios_unicos = sorted(self.df['ESCENARIO'].unique())
        
        if 'proces_simulacion' in self.df.columns:
            self.df['Tipo de Modelo'] = self.df['proces_simulacion']
        
        self.modelos = [m for m in MODELOS if m in self.df.columns]
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)
        
        # Determinar base vs diff
        if len(self.escenarios_unicos) == 2:
            e1, e2 = self.escenarios_unicos
            if ('sin' in e1.lower() or 'no' in e1.lower()) and not ('sin' in e2.lower()):
                self.esc_base, self.esc_diff = e1, e2
            elif ('sin' in e2.lower() or 'no' in e2.lower()):
                self.esc_base, self.esc_diff = e2, e1
            else:
                self.esc_base, self.esc_diff = e1, e2
        else:
            self.esc_base = self.escenarios_unicos[0]
            self.esc_diff = self.escenarios_unicos[-1]

        print(f"Base: {self.esc_base} | Comparado: {self.esc_diff}")

    def ejecutar_analisis_completo(self):
        print("1️⃣  Comparativa Global...")
        self._1_comparativo_global()
        print("2️⃣  Modelo Generador...")
        self._2_modelo_generador()
        print("3️⃣  Variabilidad IQR (Por Tipo)...")
        self._3_variabilidad_iqr()
        print("4️⃣  Distribución...")
        self._4_distribucion()
        print("5️⃣  Varianza...")
        self._5_varianza_tendencias()
        print("6️⃣  Sensibilidad Ruido...")
        self._6_sensibilidad_ruido()
        print("7️⃣  Robustez (QCD)...")
        self._7_analisis_robustez()
        print("8️⃣  Excel DM...")
        self._analisis_dm_excel()
        print("\n✅ HECHO.")

    # ========================================================================
    # 1. COMPARATIVAS GLOBALES
    # ========================================================================
    def _1_comparativo_global(self):
        datos_agg = self.df.groupby(['ESCENARIO'])[self.modelos].median().T
        
        # 1.1 Barras HORIZONTALES (Izquierda a Derecha)
        fig, ax = plt.subplots(figsize=(15, 10))
        y = np.arange(len(self.modelos))
        height = 0.35 # Altura de la barra en horizontal
        
        # Note: En barh, el primer argumento es Y, el segundo es width (valor X)
        ax.barh(y - height/2, datos_agg[self.esc_base], height, label=self.esc_base, color=COLOR_SIN_DIFF)
        ax.barh(y + height/2, datos_agg[self.esc_diff], height, label=self.esc_diff, color=COLOR_CON_DIFF)

        ax.set_xlabel('Mediana ECRPS', fontweight='bold')
        ax.set_title(f'1.1 Rendimiento: {self.esc_base} vs {self.esc_diff}', fontweight='bold')
        ax.set_yticks(y)
        ax.set_yticklabels(self.modelos)
        ax.invert_yaxis() # Para que el primer modelo esté arriba
        ax.legend()
        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_1_Comparacion_Barras_Horizontales.png', dpi=300)
        plt.close()

        # 1.2 Cambio Porcentual (Ya era Horizontal)
        cambio_pct = ((datos_agg[self.esc_diff] - datos_agg[self.esc_base]) / datos_agg[self.esc_base]) * 100
        cambio_pct = cambio_pct.sort_values()
        
        fig, ax = plt.subplots(figsize=(14, 8))
        norm = plt.Normalize(cambio_pct.min(), cambio_pct.max())
        cmap = plt.get_cmap(CMAP_HEATMAP)
        colores = [cmap(norm(v)) for v in cambio_pct.values]
        
        bars = ax.barh(cambio_pct.index, cambio_pct.values, color=colores, edgecolor='k')
        
        ax.axvline(0, color='k', linestyle='--')
        ax.set_xlabel('Cambio Porcentual del Error (%)', fontweight='bold')
        ax.set_title('1.2 Impacto de la Diferenciación (Verde = Mejora)', fontweight='bold')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.1f}%', va='center', ha=align)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_2_Cambio_Porcentual.png', dpi=300)
        plt.close()

    # ========================================================================
    # 2. MODELO GENERADOR
    # ========================================================================
    def _2_modelo_generador(self):
        if 'Tipo de Modelo' not in self.df.columns: return

        df_diff = self.df[self.df['ESCENARIO'] == self.esc_diff]
        df_base = self.df[self.df['ESCENARIO'] == self.esc_base]
        
        piv_diff = df_diff.groupby('Tipo de Modelo')[self.modelos].median()
        piv_base = df_base.groupby('Tipo de Modelo')[self.modelos].median()

        # 2.1 Heatmap Normalizado
        fig, ax = plt.subplots(figsize=(16, 9))
        row_stats = piv_diff.T.agg(['median', 'std'], axis=1)
        zscore = piv_diff.T.sub(row_stats['median'], axis=0).div(row_stats['std'].replace(0,1), axis=0)
        
        sns.heatmap(zscore, annot=True, fmt='.2f', cmap=CMAP_HEATMAP, center=0, ax=ax)
        ax.set_title(f'2.1 Z-Score Rendimiento - {self.esc_diff}', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_1_MG_ZScore_Diferenciado.png', dpi=300)
        plt.close()

        # 2.2 Cambio Absoluto
        delta = piv_diff - piv_base
        fig, ax = plt.subplots(figsize=(16, 9))
        sns.heatmap(delta.T, annot=True, fmt='.3f', cmap=CMAP_HEATMAP, center=0, ax=ax)
        ax.set_title(f'2.2 Cambio en ECRPS (Diff - Base)', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_2_MG_Cambios.png', dpi=300)
        plt.close()

    # ========================================================================
    # 3. VARIABILIDAD IQR (AGRUPADO POR TIPO)
    # ========================================================================
    def _3_variabilidad_iqr(self):
        if 'Tipo de Modelo' not in self.df.columns: return
        
        # --- PREPARACIÓN DE DATOS PARA 3.1 y 3.2 ---
        # Queremos métricas agregadas por TIPO DE MODELO
        
        tipos_modelo = self.df['Tipo de Modelo'].dropna().unique()
        data_resumen = []

        for tipo in tipos_modelo:
            # Función interna para calcular promedio de IQRs de los modelos en ese tipo
            def calcular_iqr_promedio_tipo(escenario):
                subset = self.df[(self.df['ESCENARIO'] == escenario) & 
                                 (self.df['Tipo de Modelo'] == tipo)]
                if subset.empty: return np.nan
                
                iqrs_individuales = []
                for mod in self.modelos:
                    vals = subset[mod].dropna()
                    if len(vals) > 0:
                        q75, q25 = np.percentile(vals, [75, 25])
                        iqrs_individuales.append(q75 - q25)
                
                return np.mean(iqrs_individuales) if iqrs_individuales else np.nan

            iqr_base = calcular_iqr_promedio_tipo(self.esc_base)
            iqr_diff = calcular_iqr_promedio_tipo(self.esc_diff)
            
            if not np.isnan(iqr_diff):
                delta = iqr_diff - iqr_base if not np.isnan(iqr_base) else np.nan
                data_resumen.append({
                    'Tipo': tipo, 
                    'IQR_Diff': iqr_diff,
                    'Delta_IQR': delta
                })
        
        df_resumen = pd.DataFrame(data_resumen)
        if df_resumen.empty: return

        # 3.1 Gráfico por TIPO DE MODELO - Nivel absoluto (Diff)
        df_31 = df_resumen.sort_values('IQR_Diff')
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.barh(df_31['Tipo'], df_31['IQR_Diff'], color=COLOR_CON_DIFF, alpha=0.8)
        ax.set_title(f'3.1 Variabilidad Promedio (IQR) por TIPO DE MODELO - {self.esc_diff}', fontweight='bold')
        ax.set_xlabel('IQR Promedio del Tipo')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_1_IQR_Diferenciado_PorTipo.png', dpi=300)
        plt.close()

        # 3.2 Cambio IQR por TIPO DE MODELO (Delta)
        # Aquí cumplimos el requerimiento: 3.2 ahora es sobre Tipos, no modelos individuales
        df_32 = df_resumen.dropna(subset=['Delta_IQR']).sort_values('Delta_IQR')
        
        fig, ax = plt.subplots(figsize=(12, 8))
        colors = ['green' if x < 0 else 'red' for x in df_32['Delta_IQR']]
        bars = ax.barh(df_32['Tipo'], df_32['Delta_IQR'], color=colors, alpha=0.7, edgecolor='k')
        
        ax.set_title('3.2 Cambio en Variabilidad (Delta IQR) por TIPO DE MODELO\n(Verde = Menos variabilidad con diferenciación)', fontweight='bold')
        ax.axvline(0, color='k', linestyle='--')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.3f}', va='center', ha=align)
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_2_Cambios_IQR_PorTipo.png', dpi=300)
        plt.close()

    # ========================================================================
    # 4. DISTRIBUCIÓN
    # ========================================================================
    def _4_distribucion(self):
        if 'Distribución' not in self.df.columns: return

        piv_diff = self.df[self.df['ESCENARIO'] == self.esc_diff].groupby('Distribución')[self.modelos].median()
        piv_base = self.df[self.df['ESCENARIO'] == self.esc_base].groupby('Distribución')[self.modelos].median()
        
        # 4.1 Heatmap (Diff)
        fig, ax = plt.subplots(figsize=(14, 8))
        sns.heatmap(piv_diff.T, annot=True, fmt='.3f', cmap=CMAP_HEATMAP, ax=ax)
        ax.set_title(f'4.1 Rendimiento por Distribución - {self.esc_diff}', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_1_Dist_Heatmap_Diferenciado.png', dpi=300)
        plt.close()

        # 4.2 Heatmap (Delta)
        delta = piv_diff - piv_base
        fig, ax = plt.subplots(figsize=(14, 8))
        sns.heatmap(delta.T, annot=True, fmt='.3f', cmap=CMAP_HEATMAP, center=0, ax=ax)
        ax.set_title(f'4.2 Diferencial (Diff - Base)', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_2_Dist_Heatmap_Diferencial.png', dpi=300)
        plt.close()

    # ========================================================================
    # 5. VARIANZA
    # ========================================================================
    def _5_varianza_tendencias(self):
        if 'Varianza error' not in self.df.columns: return
        
        df_diff = self.df[self.df['ESCENARIO'] == self.esc_diff]
        df_base = self.df[self.df['ESCENARIO'] == self.esc_base]

        # 5.1 Tendencias Diferenciado
        fig, ax = plt.subplots(figsize=(14, 8))
        for mod in self.modelos:
            c = df_diff.groupby('Varianza error')[mod].median()
            ax.plot(c.index, c.values, marker='o', label=mod)
        ax.set_title(f'5.1 Sensibilidad a Varianza - {self.esc_diff}', fontweight='bold')
        ax.legend(bbox_to_anchor=(1.01, 1))
        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_1_Varianza_Tendencias_Diferenciado.png', dpi=300)
        plt.close()

        # 5.2 Cambio
        fig, ax = plt.subplots(figsize=(14, 8))
        for mod in self.modelos:
            c1 = df_diff.groupby('Varianza error')[mod].median()
            c2 = df_base.groupby('Varianza error')[mod].median()
            if not c1.empty and not c2.empty:
                delta = c1 - c2
                ax.plot(delta.index, delta.values, marker='o', label=mod)
        ax.axhline(0, color='k', linestyle='--')
        ax.set_title('5.2 Cambio de Comportamiento (Negativo = Mejora)', fontweight='bold')
        ax.legend(bbox_to_anchor=(1.01, 1))
        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_2_Varianza_Cambio_Comportamiento.png', dpi=300)
        plt.close()

    # ========================================================================
    # 6. SENSIBILIDAD RUIDO
    # ========================================================================
    def _6_sensibilidad_ruido(self):
        if 'Varianza error' not in self.df.columns: return

        slopes = []
        for esc in [self.esc_base, self.esc_diff]:
            df_c = self.df[self.df['ESCENARIO'] == esc]
            for mod in self.modelos:
                dat = df_c[['Varianza error', mod]].dropna()
                if len(dat) > 2:
                    res = stats.theilslopes(dat[mod], dat['Varianza error'])
                    slopes.append({'Modelo': mod, 'Esc': esc, 'Slope': res[0]})
        
        df_slopes = pd.DataFrame(slopes)
        if df_slopes.empty: return

        # 6.1 Sensibilidad Diff (Ya era Horizontal)
        sub = df_slopes[df_slopes['Esc'] == self.esc_diff].sort_values('Slope')
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.barh(sub['Modelo'], sub['Slope'], color=COLOR_CON_DIFF)
        ax.set_title(f'6.1 Sensibilidad Ruido (Pendiente) - {self.esc_diff}', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_1_Sensibilidad_Ruido_Diferenciado.png', dpi=300)
        plt.close()

        # 6.2 Cambio Sensibilidad (Ya era Horizontal)
        piv = df_slopes.pivot(index='Modelo', columns='Esc', values='Slope')
        piv['Delta'] = piv[self.esc_diff] - piv[self.esc_base]
        piv = piv.sort_values('Delta')
        
        fig, ax = plt.subplots(figsize=(12, 8))
        colors = ['green' if x < 0 else 'red' for x in piv['Delta']]
        bars = ax.barh(piv.index, piv['Delta'], color=colors, edgecolor='k')
        ax.axvline(0, color='k')
        ax.set_title('6.2 Cambio en Sensibilidad (Verde = Menos sensible)', fontweight='bold')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.4f}', va='center', ha=align)
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_2_Sensibilidad_Cambio.png', dpi=300)
        plt.close()

    # ========================================================================
    # 7. ROBUSTEZ (QCD) - CON COMPARACIÓN
    # ========================================================================
    def _7_analisis_robustez(self):
        # Calcular QCD para ambos escenarios
        metricas = []
        for esc in [self.esc_base, self.esc_diff]:
            df_c = self.df[self.df['ESCENARIO'] == esc]
            for mod in self.modelos:
                dat = df_c[mod].dropna()
                if len(dat) > 0:
                    q75, q25 = np.percentile(dat, [75, 25])
                    if (q75 + q25) > 0:
                        qcd = (q75 - q25) / (q75 + q25)
                        metricas.append({'Modelo': mod, 'Esc': esc, 'QCD': qcd})
        
        df_qcd = pd.DataFrame(metricas)
        if df_qcd.empty: return

        # 7.1 QCD Diferenciado (Ya era Horizontal)
        sub_diff = df_qcd[df_qcd['Esc'] == self.esc_diff].sort_values('QCD')
        fig, ax = plt.subplots(figsize=(12, 8))
        norm = plt.Normalize(sub_diff['QCD'].min(), sub_diff['QCD'].max())
        cmap = plt.get_cmap(CMAP_HEATMAP)
        colors = [cmap(1 - norm(v)) for v in sub_diff['QCD']]

        bars = ax.barh(sub_diff['Modelo'], sub_diff['QCD'], color=colors, edgecolor='k')
        ax.set_title(f'7.1 Robustez (QCD) - {self.esc_diff} (Menor es mejor)', fontweight='bold')
        
        for bar in bars:
            w = bar.get_width()
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.3f}', va='center', ha='left')
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '7_1_Robustez_Diferenciado.png', dpi=300)
        plt.close()

        # 7.2 COMPARACIÓN DE ROBUSTEZ (Ya era Horizontal)
        piv = df_qcd.pivot(index='Modelo', columns='Esc', values='QCD')
        piv['Delta'] = piv[self.esc_diff] - piv[self.esc_base]
        piv = piv.sort_values('Delta')
        
        fig, ax = plt.subplots(figsize=(12, 8))
        colors = ['green' if x < 0 else 'red' for x in piv['Delta']]
        bars = ax.barh(piv.index, piv['Delta'], color=colors, edgecolor='k', alpha=0.8)
        
        ax.set_title('7.2 Cambio en Robustez (QCD)\n(Valores negativos indican mayor robustez en diferenciado)', fontweight='bold')
        ax.axvline(0, color='k', linestyle='--')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.3f}', va='center', ha=align)
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '7_2_Robustez_Cambio.png', dpi=300)
        plt.close()

    # ========================================================================
    # 8. EXCEL
    # ========================================================================
    def _analisis_dm_excel(self):
        res = [comparar_modelo_entre_escenarios(self.df, m, self.esc_base, self.esc_diff) for m in self.modelos]
        df_res = pd.DataFrame(res)
        nombre = self.dir_salida / "Comparacion_DM_Por_Modelo.xlsx"
        df_res.to_excel(nombre, index=False)
        print(f"   ✅ Excel: {nombre}")

# ============================================================================
# MAIN
# ============================================================================
def main():
    try:
        AnalizadorBaseCompleta(RUTA_DATOS).ejecutar_analisis_completo()
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


INICIANDO ANÁLISIS COMPARATIVO (AJUSTADO - BARRAS HORIZONTALES)

Base: Sin diferenciación | Comparado: Diferenciado
1️⃣  Comparativa Global...
2️⃣  Modelo Generador...
3️⃣  Variabilidad IQR (Por Tipo)...
4️⃣  Distribución...
5️⃣  Varianza...
6️⃣  Sensibilidad Ruido...
7️⃣  Robustez (QCD)...
8️⃣  Excel DM...
   ✅ Excel: resultados_escenarios_comparativos\Comparacion_DM_Por_Modelo.xlsx

✅ HECHO.


# Aumento d ARIMA

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from scipy.interpolate import UnivariateSpline
import warnings

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./datos/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
DIR_SALIDA = "./resultados_diff_d"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Paleta de colores para modelos
COLORES_MODELOS = plt.cm.tab10(np.linspace(0, 1, len(MODELOS)))
COLOR_MAP_MODELOS = {mod: COLORES_MODELOS[i] for i, mod in enumerate(MODELOS)}

# Colores para modalidades
COLOR_SIN_DIFF = '#e74c3c'  # Rojo
COLOR_CON_DIFF = '#3498db'  # Azul

CMAP_HEATMAP = 'RdYlGn_r' # Rojo = Malo (Alto Error), Verde = Bueno (Bajo Error)

# ============================================================================
# CLASE PRINCIPAL
# ============================================================================

class AnalizadorSensibilidadD:
    
    def __init__(self, ruta_datos):
        print("\n" + "=" * 80)
        print("ANÁLISIS DE SENSIBILIDAD AL PARÁMETRO D (ORDEN DE DIFERENCIACIÓN)")
        print("=" * 80 + "\n")
        
        self.df = pd.read_excel(ruta_datos)
        
        # Limpieza
        self.df['Modalidad'] = self.df['Modalidad'].astype(str).str.strip().str.upper()
        self.df['d'] = pd.to_numeric(self.df['d'], errors='coerce')
        
        # Filtrar datos válidos
        self.df = self.df[self.df['d'].notna()].copy()
        self.valores_d = sorted(self.df['d'].unique())
        
        self.modelos = [m for m in MODELOS if m in self.df.columns]
        self.modalidades = sorted(self.df['Modalidad'].unique())
        
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)
        
        print(f"📊 Valores de d: {self.valores_d}")
        print(f"🎭 Modalidades: {self.modalidades}")
        print(f"📈 Modelos analizados: {len(self.modelos)}\n")

    def _fmt(self, x):
        """
        Formatea números: 
        - Usa notación científica si abs(x) >= 10,000 (5 dígitos) o x < 0.001
        - Usa decimales estándar en caso contrario.
        """
        if pd.isna(x):
            return ""
        if x == 0:
            return "0"
        # Umbral: 5 dígitos enteros (10,000) o muy pequeños
        if abs(x) >= 10000 or (0 < abs(x) < 0.001):
            return f"{x:.2e}"
        return f"{x:.3f}"

    def ejecutar_analisis_completo(self):
        """Ejecuta todas las preguntas de investigación"""
        
        # --- VISUALIZACIÓN GENERAL ---
        self._visualizar_heatmaps_ecrps_absolutos()

        print("\n" + "=" * 80)
        print("PREGUNTA 1: ¿Qué modelo es más sensible a los cambios en d?")
        print("=" * 80)
        self._pregunta1_sensibilidad_modelos()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 2: ¿Existe un punto de inflexión en d?")
        print("=" * 80)
        self._pregunta2_punto_inflexion()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 3: ¿Cómo impacta d en la variabilidad?")
        print("=" * 80)
        self._pregunta3_variabilidad()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 4: ¿La diferenciación previa amplifica el efecto de d?")
        print("=" * 80)
        self._pregunta4_interaccion_modalidad()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 5: ¿Cuándo es significativa la diferenciación? (Foco: Sieve Bootstrap)")
        print("=" * 80)
        self._pregunta5_consistencia()
        
        print("\n✅ ANÁLISIS COMPLETO FINALIZADO")

    # ========================================================================
    # VISUALIZACIÓN EXTRA: HEATMAPS DE ECRPS MEDIO (Modelo vs d)
    # ========================================================================
    def _visualizar_heatmaps_ecrps_absolutos(self):
        print("📊 Generando Heatmaps Generales de Rendimiento...")

        fig, axes = plt.subplots(1, 2, figsize=(24, 10))
        
        vmin = self.df[self.modelos].min().min()
        vmax_robust = np.percentile(self.df[self.modelos].values, 95)

        for idx, modalidad in enumerate(self.modalidades):
            if idx >= 2: break 
            
            ax = axes[idx]
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            
            heatmap_data = df_mod.groupby('d')[self.modelos].mean().T
            
            # Crear matriz de anotaciones formateadas
            annot_data = heatmap_data.applymap(self._fmt)
            
            sns.heatmap(heatmap_data, ax=ax, cmap=CMAP_HEATMAP, 
                        annot=annot_data.values, fmt='', # fmt='' es necesario cuando pasamos strings
                        vmin=vmin, vmax=vmax_robust, 
                        linewidths=.5, cbar_kws={'label': 'ECRPS Medio'})
            
            ax.set_title(f'Rendimiento Medio (ECRPS) - {modalidad}', fontweight='bold', fontsize=14)
            ax.set_xlabel('Orden de Diferenciación (d)', fontsize=12, fontweight='bold')
            ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
            ax.tick_params(axis='y', rotation=0)

        plt.suptitle('Comparación de ECRPS Medio: Modelos vs. Orden de Diferenciación (d)', 
                     fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        nombre_archivo = "0_General_Heatmaps_ECRPS_Medio.png"
        plt.savefig(self.dir_salida / nombre_archivo, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅ Gráfico guardado: {nombre_archivo}")

    # ========================================================================
    # PREGUNTA 1: SENSIBILIDAD DE MODELOS A d
    # ========================================================================
    def _pregunta1_sensibilidad_modelos(self):
        resultados = []
        for modalidad in self.modalidades:
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            for modelo in self.modelos:
                serie = df_mod.groupby('d')[modelo].median()
                if len(serie) > 3:
                    slope, intercept, _, _ = stats.theilslopes(serie.values, serie.index)
                    corr, p_value = stats.spearmanr(serie.index, serie.values)
                    rango = serie.max() - serie.min()
                    variacion_pct = (rango / serie.mean()) * 100 if serie.mean() != 0 else 0
                    
                    resultados.append({
                        'Modelo': modelo, 'Modalidad': modalidad, 'Pendiente': slope,
                        'Correlación': corr, 'p_value': p_value, 'Rango': rango,
                        'Variación_%': variacion_pct, 'Sensibilidad_Score': abs(slope) * abs(corr)
                    })
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P1_Sensibilidad_Modelos.xlsx", index=False)
        print(f"   ✅ Excel guardado: P1_Sensibilidad_Modelos.xlsx")
        
        self._visualizar_pregunta1(df_resultados)
        
        print("\n🔥 TOP 3 MODELOS MÁS SENSIBLES A d:")
        top_sensibles = df_resultados.nlargest(3, 'Sensibilidad_Score')
        for idx, row in top_sensibles.iterrows():
            print(f"   {row['Modelo']} ({row['Modalidad']}): Score={self._fmt(row['Sensibilidad_Score'])}")

    def _visualizar_pregunta1(self, df_resultados):
        # 1.1 Ranking
        fig, axes = plt.subplots(1, 2, figsize=(18, 8))
        for idx, modalidad in enumerate(self.modalidades):
            if idx >= 2: break
            df_sub = df_resultados[df_resultados['Modalidad'] == modalidad].sort_values('Sensibilidad_Score')
            ax = axes[idx]
            colors = [COLOR_MAP_MODELOS[m] for m in df_sub['Modelo']]
            bars = ax.barh(df_sub['Modelo'], df_sub['Sensibilidad_Score'], color=colors, edgecolor='black', alpha=0.8)
            ax.set_xlabel('Score de Sensibilidad', fontweight='bold')
            ax.set_title(f'P1.1: Sensibilidad a d - {modalidad}', fontweight='bold')
            ax.grid(axis='x', alpha=0.3)
            
            # Anotación formateada
            for bar in bars:
                width = bar.get_width()
                label = self._fmt(width)
                ax.text(width, bar.get_y() + bar.get_height()/2, label, va='center', fontsize=9)

        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_1_Ranking_Sensibilidad.png', dpi=300)
        plt.close()
        
        # 1.2 Comparación
        fig, ax = plt.subplots(figsize=(14, 8))
        piv = df_resultados.pivot(index='Modelo', columns='Modalidad', values='Pendiente')
        piv.plot(kind='bar', ax=ax, width=0.7, edgecolor='black', 
                 color=[COLOR_SIN_DIFF, COLOR_CON_DIFF] if len(piv.columns)==2 else None)
        ax.set_ylabel('Pendiente (ECRPS vs d)')
        ax.set_title('P1.2: Pendientes de Sensibilidad por Modalidad')
        ax.axhline(0, color='black', linestyle='--')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_2_Pendientes_Comparacion.png', dpi=300)
        plt.close()

    # ========================================================================
    # PREGUNTA 2: PUNTO DE INFLEXIÓN
    # ========================================================================
    def _pregunta2_punto_inflexion(self):
        resultados = []
        for modalidad in self.modalidades:
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            for modelo in self.modelos:
                serie = df_mod.groupby('d')[modelo].median().dropna()
                if len(serie) >= 5:
                    x, y = serie.index.values, serie.values
                    try:
                        spline = UnivariateSpline(x, y, s=0.1, k=3)
                        x_fine = np.linspace(x.min(), x.max(), 100)
                        y_second_deriv = spline.derivative(n=2)(x_fine)
                        d_inflexion = x_fine[np.argmax(np.abs(y_second_deriv))]
                        resultados.append({
                            'Modelo': modelo, 'Modalidad': modalidad, 'd_Inflexión': round(d_inflexion, 1),
                            'Cambio_Total_%': ((serie.iloc[-1] - serie.iloc[0]) / serie.iloc[0] * 100) if serie.iloc[0] != 0 else 0
                        })
                    except: pass
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P2_Puntos_Inflexion.xlsx", index=False)
        print(f"   ✅ Excel guardado: P2_Puntos_Inflexion.xlsx")
        self._visualizar_pregunta2(df_resultados)

    def _visualizar_pregunta2(self, df_resultados):
        fig, ax = plt.subplots(figsize=(14, 8))
        for modalidad in self.modalidades:
            df_sub = df_resultados[df_resultados['Modalidad'] == modalidad]
            color = COLOR_SIN_DIFF if 'SIN' in modalidad else COLOR_CON_DIFF
            ax.scatter(df_sub['d_Inflexión'], df_sub['Modelo'], s=200, alpha=0.7, color=color, edgecolor='black', label=modalidad, marker='D')
        ax.axvline(df_resultados['d_Inflexión'].mean(), color='gray', linestyle='--', label='Media Global')
        ax.set_xlabel('Valor de d en Punto de Inflexión')
        ax.set_title('P2.1: Distribución de Puntos de Inflexión')
        ax.legend()
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P2_1_Distribucion_Inflexion.png', dpi=300)
        plt.close()

    # ========================================================================
    # PREGUNTA 3: IMPACTO EN VARIABILIDAD
    # ========================================================================
    def _pregunta3_variabilidad(self):
        resultados = []
        for modalidad in self.modalidades:
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            for d_val in self.valores_d:
                df_d = df_mod[df_mod['d'] == d_val]
                for modelo in self.modelos:
                    datos = df_d[modelo].dropna()
                    if len(datos) > 5:
                        q75, q25 = np.percentile(datos, [75, 25])
                        resultados.append({
                            'Modelo': modelo, 'Modalidad': modalidad, 'd': d_val,
                            'IQR': q75 - q25,
                            'QCD': (q75 - q25) / (q75 + q25) if (q75 + q25) > 0 else 0
                        })
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P3_Variabilidad_por_d.xlsx", index=False)
        print(f"   ✅ Excel guardado: P3_Variabilidad_por_d.xlsx")
        self._visualizar_pregunta3(df_resultados)

    def _visualizar_pregunta3(self, df_resultados):
        fig, axes = plt.subplots(1, 2, figsize=(18, 7))
        for idx, modalidad in enumerate(self.modalidades):
            if idx >= 2: break
            df_mod = df_resultados[df_resultados['Modalidad'] == modalidad]
            ax = axes[idx]
            for modelo in self.modelos:
                df_m = df_mod[df_mod['Modelo'] == modelo]
                if not df_m.empty:
                    ax.plot(df_m['d'], df_m['IQR'], marker='o', label=modelo, color=COLOR_MAP_MODELOS[modelo])
            ax.set_title(f'P3.1: Evolución IQR - {modalidad}')
            ax.set_ylabel('IQR')
            ax.set_xlabel('d')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P3_1_Evolucion_IQR.png', dpi=300)
        plt.close()
        
        # Heatmap QCD
        for modalidad in self.modalidades:
            piv = df_resultados[df_resultados['Modalidad'] == modalidad].pivot(index='Modelo', columns='d', values='QCD')
            
            # Formatear anotaciones
            annot_qcd = piv.applymap(self._fmt)
            
            fig, ax = plt.subplots(figsize=(12, 6))
            sns.heatmap(piv, annot=annot_qcd.values, fmt='', cmap=CMAP_HEATMAP, ax=ax)
            ax.set_title(f'P3.2: Robustez Relativa (QCD) - {modalidad}')
            plt.tight_layout()
            plt.savefig(self.dir_salida / f'P3_2_Heatmap_QCD_{modalidad}.png', dpi=300)
            plt.close()

    # ========================================================================
    # PREGUNTA 4: INTERACCIÓN MODALIDAD × d
    # ========================================================================
    def _pregunta4_interaccion_modalidad(self):
        if len(self.modalidades) < 2: return
        resultados = []
        for modelo in self.modelos:
            pendientes = {}
            for modalidad in self.modalidades:
                df_mod = self.df[self.df['Modalidad'] == modalidad]
                serie = df_mod.groupby('d')[modelo].median()
                if len(serie) > 3:
                    slope, _, _, _ = stats.theilslopes(serie.values, serie.index)
                    pendientes[modalidad] = slope
            
            if len(pendientes) == 2:
                mod1, mod2 = self.modalidades
                interaccion = pendientes[mod2] - pendientes[mod1]
                resultados.append({
                    'Modelo': modelo, 'Pendiente_Diff': pendientes.get(mod2,0), 'Pendiente_Base': pendientes.get(mod1,0),
                    'Interacción': interaccion,
                    'Interpretación': 'Amplifica' if abs(pendientes[mod2]) > abs(pendientes[mod1]) else 'Modera'
                })
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P4_Interaccion_Modalidad.xlsx", index=False)
        print(f"   ✅ Excel guardado: P4_Interaccion_Modalidad.xlsx")
        self._visualizar_pregunta4(df_resultados)

    def _visualizar_pregunta4(self, df_resultados):
        df_sorted = df_resultados.sort_values('Interacción')
        fig, ax = plt.subplots(figsize=(14, 8))
        colors = ['#27ae60' if x < 0 else '#e74c3c' for x in df_sorted['Interacción']]
        bars = ax.barh(df_sorted['Modelo'], df_sorted['Interacción'], color=colors, edgecolor='black', alpha=0.8)
        ax.axvline(0, color='black', linestyle='--')
        ax.set_xlabel('Efecto de Interacción (Pendiente_Diff - Pendiente_Base)')
        ax.set_title('P4.2: Efecto de la Diferenciación sobre Sensibilidad a d')
        
        for bar in bars:
            width = bar.get_width()
            label = self._fmt(width)
            ax.text(width, bar.get_y() + bar.get_height()/2, label, 
                   ha='left' if width > 0 else 'right', va='center', fontsize=9, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P4_2_Efecto_Interaccion.png', dpi=300)
        plt.close()

    # ========================================================================
    # PREGUNTA 5: SIGNIFICANCIA DE LA DIFERENCIACIÓN (SIEVE BOOTSTRAP)
    # ========================================================================
    def _pregunta5_consistencia(self):
        print(f"\n🔎 Analizando significancia estadística (Mann-Whitney U)...")
        resultados = []
        if len(self.modalidades) < 2:
            print("   ⚠️  No es posible comparar (falta modalidad).")
            return

        mod_sin = [m for m in self.modalidades if 'SIN' in m][0]
        mod_con = [m for m in self.modalidades if 'CON' in m][0]

        for modelo in self.modelos:
            for d_val in self.valores_d:
                datos_sin = self.df[(self.df['Modalidad'] == mod_sin) & (self.df['d'] == d_val)][modelo].dropna()
                datos_con = self.df[(self.df['Modalidad'] == mod_con) & (self.df['d'] == d_val)][modelo].dropna()
                
                if len(datos_sin) > 3 and len(datos_con) > 3:
                    stat, p_value = stats.mannwhitneyu(datos_sin, datos_con, alternative='two-sided')
                    diff_mediana = datos_sin.median() - datos_con.median()
                    resultados.append({
                        'Modelo': modelo, 'd': d_val, 'p_value': p_value,
                        'Significativo': p_value < 0.05, 'Diff_Mediana': diff_mediana
                    })

        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P5_Significancia_Diferenciacion.xlsx", index=False)
        print(f"   ✅ Excel guardado: P5_Significancia_Diferenciacion.xlsx")
        
        self._visualizar_pregunta5(df_resultados)
        
        print("\n🧐 ANÁLISIS ESPECÍFICO: Sieve Bootstrap")
        df_sb = df_resultados[df_resultados['Modelo'] == 'Sieve Bootstrap'].sort_values('d')
        if not df_sb.empty:
            significativos = df_sb[df_sb['Significativo']]
            if not significativos.empty:
                primer_d = significativos['d'].iloc[0]
                print(f"   👉 Para Sieve Bootstrap, la diferenciación es estadísticamente significativa (p < 0.05)")
                print(f"      a partir de d = {primer_d}")
            else:
                print("   👉 No se encontraron diferencias significativas para Sieve Bootstrap en ningún d.")
        else:
            print("   ⚠️  Sieve Bootstrap no encontrado en los resultados.")

    def _visualizar_pregunta5(self, df_resultados):
        piv_p = df_resultados.pivot(index='Modelo', columns='d', values='p_value')
        log_p = -np.log10(piv_p + 1e-10) 
        
        # Formatear P-values para anotación
        annot_p = piv_p.applymap(self._fmt)

        fig, ax = plt.subplots(figsize=(14, 8))
        # Usamos 'magma' en minúsculas para evitar KeyError
        sns.heatmap(log_p, cmap='magma', annot=annot_p.values, fmt='', 
                    cbar_kws={'label': '-log10(p-value)'}, ax=ax)
        
        ax.set_title('P5.1: Significancia Estadística (p-values) - Valores claros = Muy Significativo')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P5_1_Heatmap_Significancia.png', dpi=300)
        plt.close()
        
        # 5.2 Evolución P-value
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.axhline(0.05, color='red', linestyle='--', label='p=0.05')
        
        df_sb = df_resultados[df_resultados['Modelo'] == 'Sieve Bootstrap']
        if not df_sb.empty:
            ax.plot(df_sb['d'], df_sb['p_value'], marker='o', linewidth=3, color='#8e44ad', label='Sieve Bootstrap')
        
        for modelo in self.modelos:
            if modelo != 'Sieve Bootstrap':
                df_m = df_resultados[df_resultados['Modelo'] == modelo]
                ax.plot(df_m['d'], df_m['p_value'], color='gray', alpha=0.15)
        
        ax.set_yscale('log')
        ax.set_ylabel('P-value (Log)')
        ax.set_title('P5.2: Evolución de P-value para Sieve Bootstrap')
        ax.legend()
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P5_2_Sieve_Bootstrap_Significancia.png', dpi=300)
        plt.close()

# ============================================================================
# EJECUCIÓN
# ============================================================================

if __name__ == "__main__":
    if Path(RUTA_DATOS).exists():
        analizador = AnalizadorSensibilidadD(RUTA_DATOS)
        analizador.ejecutar_analisis_completo()
    else:
        print(f"\n⛔ ERROR: No se encontró el archivo: {RUTA_DATOS}")


ANÁLISIS DE SENSIBILIDAD AL PARÁMETRO D (ORDEN DE DIFERENCIACIÓN)

📊 Valores de d: [np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(7), np.int64(10)]
🎭 Modalidades: ['CON_DIFF', 'SIN_DIFF']
📈 Modelos analizados: 9

📊 Generando Heatmaps Generales de Rendimiento...
   ✅ Gráfico guardado: 0_General_Heatmaps_ECRPS_Medio.png

PREGUNTA 1: ¿Qué modelo es más sensible a los cambios en d?
   ✅ Excel guardado: P1_Sensibilidad_Modelos.xlsx

🔥 TOP 3 MODELOS MÁS SENSIBLES A d:
   Block Bootstrapping (SIN_DIFF): Score=8.23e+10
   AREPD (SIN_DIFF): Score=8.04e+10
   DeepAR (SIN_DIFF): Score=7.69e+10

PREGUNTA 2: ¿Existe un punto de inflexión en d?
   ✅ Excel guardado: P2_Puntos_Inflexion.xlsx

PREGUNTA 3: ¿Cómo impacta d en la variabilidad?
   ✅ Excel guardado: P3_Variabilidad_por_d.xlsx

PREGUNTA 4: ¿La diferenciación previa amplifica el efecto de d?
   ✅ Excel guardado: P4_Interaccion_Modalidad.xlsx

PREGUNTA 5: ¿Cuándo es significativa la diferenciación? (Foco: Sieve Bootstrap)

🔎 An

# Analisis cambio de entrenamiento

## Pre-procesamiento

In [15]:
import pandas as pd
import numpy as np

# Leer los tres archivos
arma_df = pd.read_excel("./datos/resultados_TAMANOS_CRECIENTES_ARMA.xlsx")
arima_df = pd.read_excel("./datos/resultados_TAMANOS_CRECIENTES_ARIMA.xlsx")
setar_df = pd.read_excel("./datos/resultados_TAMANOS_CRECIENTES_SETAR.xlsx")

# Filtrar los que no tienen "Promedio" en la columna "Paso"
arma_df = arma_df[arma_df['Paso'] != 'Promedio']
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
setar_df = setar_df[setar_df['Paso'] != 'Promedio']

# Lista de modelos (columnas a promediar)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario (de la columna del modelo)
    arma_promedio = arma_df[modelo].mean() if modelo in arma_df.columns else np.nan
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    setar_promedio = setar_df[modelo].mean() if modelo in setar_df.columns else np.nan
    
    fila['ARMA'] = arma_promedio
    fila['ARIMA'] = arima_promedio
    fila['SETAR'] = setar_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARMA': arma_promedio,
        'ARIMA': arima_promedio,
        'SETAR': setar_promedio
    }
    
    # Filtrar NaN si existen
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)

# Redondear valores para mejor visualización
columnas_numericas = ['ARMA', 'ARIMA', 'SETAR']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")

# Guardar tabla comparativa en Excel
tabla_comparativa.to_excel("Tabla_Comparativa_Modelos_tamaño.xlsx", index=False)
print("Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_tamaño.xlsx'")

# Procesamiento especial para SETAR
if 'Descripción' in setar_df.columns:
    setar_df = setar_df.drop('Descripción', axis=1)

# Agregar columna ESCENARIO a cada DataFrame antes de concatenar
arma_df['ESCENARIO'] = 'Lineal - estacionario'
arima_df['ESCENARIO'] = 'Lineal - NO estacionario'
setar_df['ESCENARIO'] = 'NO lineal - estacionario'

# Concatenar los tres dataframes
base_consolidada = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Guardar en un archivo Excel
base_consolidada.to_excel("Base_Tamaño_3_escenarios.xlsx", index=False)

print("\nArchivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!")
print(f"\nTotal de filas: {len(base_consolidada)}")
print(f"- ARMA: {len(arma_df)} filas")
print(f"- ARIMA: {len(arima_df)} filas")
print(f"- SETAR: {len(setar_df)} filas")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARMA   ARIMA  SETAR Mejor_Escenario
              AREPD 0.9345 13.0489 0.6971           SETAR
            AV-MCPS 0.6768  3.5050 0.6531           SETAR
Block Bootstrapping 0.9049 15.1183 0.6318           SETAR
             DeepAR 0.5650  3.6998 0.5845            ARMA
         EnCQR-LSTM 0.9515  5.9330 0.8404           SETAR
               LSPM 0.7689  1.0868 0.6586           SETAR
              LSPMW 0.7931  1.0870 0.6754           SETAR
               MCPS 0.6496  3.2780 0.6325           SETAR
    Sieve Bootstrap 0.5541  0.5583 0.6254            ARMA

Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_tamaño.xlsx'

Archivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!

Total de filas: 126000
- ARMA: 42000 filas
- ARIMA: 42000 filas
- SETAR: 42000 filas


## Analisis general

In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from itertools import combinations
import warnings
import gc

# Ignorar advertencias
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Crear carpeta de resultados
output_dir = Path("./resultados_analisis_ntotal")
output_dir.mkdir(parents=True, exist_ok=True)

# Configuración
archivo_excel = "./Base_Tamaño_3_escenarios.xlsx"
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Cargar datos
print("Cargando datos...")
try:
    df = pd.read_excel(archivo_excel)
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo '{archivo_excel}'.")
    exit()

print(f"Columnas disponibles: {df.columns.tolist()}")
print(f"N_Total únicos: {sorted(df['N_Total'].unique())}")
print(f"Número de observaciones: {len(df)}")

# ============================================================================
# FUNCIÓN DIEBOLD-MARIANO
# ============================================================================

def diebold_mariano_test(series1, series2):
    """
    Test de Diebold-Mariano.
    Asumimos que las series ingresadas ya son las pérdidas (ECRPS), 
    por lo que comparamos d = L1 - L2 directamente o cuadrática según se desee.
    Aquí se mantiene la lógica cuadrática estándar del test sobre la diferencia.
    """
    d = series1**2 - series2**2
    d = d.dropna()
    
    if len(d) < 2:
        return np.nan, np.nan
    
    d_mean = d.mean()
    n = len(d)
    d_var = d.var() / n
    
    if d_var <= 0:
        return np.nan, np.nan
    
    dm_stat = d_mean / np.sqrt(d_var)
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    
    return dm_stat, p_value

# ============================================================================
# ANÁLISIS 1: HEATMAPS - PROMEDIO DE ECRPS POR TIPO_PROCESO Y N_TOTAL
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 1: HEATMAPS - Promedio de ECRPS por Proceso y N_Total")
print("="*80)

for modelo in MODELOS:
    # Calcular promedio de ECRPS (asumiendo que el valor en excel ya es el score positivo)
    pivot_data = df.groupby(['Proceso', 'N_Total'])[modelo].mean().reset_index()
    pivot_table = pivot_data.pivot(index='Proceso', columns='N_Total', values=modelo)
    
    # Crear heatmap
    fig, ax = plt.subplots(figsize=(14, 6))
    
    sns.heatmap(pivot_table, 
                annot=True, 
                fmt='.2f',  # <--- CAMBIO: Solo 2 dígitos
                cmap='RdYlGn_r',  # Rojo=alto ECRPS (malo), Verde=bajo ECRPS (bueno)
                cbar_kws={'label': 'ECRPS Promedio', 'shrink': 0.7}, # Barra reducida al 70%
                linewidths=0.5,
                linecolor='gray',
                ax=ax)
    
    plt.title(f'Heatmap: {modelo}\nECRPS Promedio por Proceso y N_Total',
              fontsize=14, fontweight='bold', pad=20)
    plt.xlabel('N_Total (Tamaño de Muestra)', fontsize=11)
    plt.ylabel('Tipo de Proceso', fontsize=11)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    archivo_heatmap = output_dir / f"heatmap_{modelo.replace(' ', '_')}.png"
    plt.savefig(archivo_heatmap, dpi=300, bbox_inches='tight')
    print(f"Heatmap guardado: {archivo_heatmap}")
    plt.close()

# ============================================================================
# ANÁLISIS 2: MEJOR N_TOTAL POR TIPO_PROCESO Y MODELO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 2: MEJOR N_TOTAL (Menor ECRPS Promedio)")
print("="*80)

resultados_mejor_ntotal = []

for tipo_proceso in df['Tipo_Proceso'].unique():
    for modelo in MODELOS:
        df_filtrado = df[df['Tipo_Proceso'] == tipo_proceso]
        
        # Calcular ECRPS promedio por N_Total
        promedios = df_filtrado.groupby('N_Total')[modelo].mean()
        
        mejor_ntotal = promedios.idxmin()
        mejor_score = promedios.min()
        
        resultados_mejor_ntotal.append({
            'Tipo_Proceso': tipo_proceso,
            'Modelo': modelo,
            'Mejor_N_Total': mejor_ntotal,
            'ECRPS_Promedio': round(mejor_score, 4)
        })
        
        print(f"{tipo_proceso} | {modelo}: N_Total={mejor_ntotal} (ECRPS={mejor_score:.4f})")

# Guardar en Excel
df_mejor_ntotal = pd.DataFrame(resultados_mejor_ntotal)
archivo_mejor = output_dir / "mejor_ntotal_por_modelo_y_tipo.xlsx"
df_mejor_ntotal.to_excel(archivo_mejor, index=False)

# ============================================================================
# ANÁLISIS 3: COMPARACIÓN ENTRE N_TOTALES CON DIEBOLD-MARIANO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 3: Comparaciones DM entre N_Totales")
print("="*80)

resultados_dm = []

for tipo_proceso in df['Tipo_Proceso'].unique():
    for modelo in MODELOS:
        df_filtrado = df[df['Tipo_Proceso'] == tipo_proceso]
        ntotales_disponibles = sorted(df_filtrado['N_Total'].unique())
        
        for ntotal1, ntotal2 in combinations(ntotales_disponibles, 2):
            series1 = df_filtrado[df_filtrado['N_Total'] == ntotal1][modelo]
            series2 = df_filtrado[df_filtrado['N_Total'] == ntotal2][modelo]
            
            if len(series1) > 0 and len(series2) > 0:
                dm_stat, p_value = diebold_mariano_test(series1, series2)
                
                mean1 = series1.mean()
                mean2 = series2.mean()
                
                es_significativo = p_value < 0.05 if not np.isnan(p_value) else False
                
                if es_significativo:
                    if mean1 < mean2:
                        ganador = ntotal1
                        diferencia = "N_Total={} es significativamente MEJOR".format(ntotal1)
                    else:
                        ganador = ntotal2
                        diferencia = "N_Total={} es significativamente MEJOR".format(ntotal2)
                else:
                    ganador = None
                    diferencia = "Sin diferencia significativa"
                
                resultados_dm.append({
                    'Tipo_Proceso': tipo_proceso,
                    'Modelo': modelo,
                    'N_Total_1': ntotal1,
                    'N_Total_2': ntotal2,
                    'ECRPS_1': round(mean1, 4),
                    'ECRPS_2': round(mean2, 4),
                    'P_Value': round(p_value, 4) if not np.isnan(p_value) else None,
                    'Significativo': 'Sí' if es_significativo else 'No',
                    'Interpretación': diferencia
                })

df_dm = pd.DataFrame(resultados_dm)
archivo_dm = output_dir / "comparaciones_ntotal_diebold_mariano.xlsx"
with pd.ExcelWriter(archivo_dm, engine='openpyxl') as writer:
    df_dm.to_excel(writer, sheet_name='Resultados', index=False)

# ============================================================================
# ANÁLISIS 4: RESUMEN ESTADÍSTICO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 4: Resumen Estadístico (ECRPS)")
print("="*80)

resumen_stats = []
for tipo_proceso in df['Tipo_Proceso'].unique():
    for ntotal in sorted(df['N_Total'].unique()):
        df_filtrado = df[(df['Tipo_Proceso'] == tipo_proceso) & (df['N_Total'] == ntotal)]
        
        for modelo in MODELOS:
            vals = df_filtrado[modelo] # Asumimos ECRPS directo
            resumen_stats.append({
                'Tipo_Proceso': tipo_proceso,
                'N_Total': ntotal,
                'Modelo': modelo,
                'Media': round(vals.mean(), 4),
                'Mediana': round(vals.median(), 4),
                'Desv_Std': round(vals.std(), 4),
                'Min': round(vals.min(), 4),
                'Max': round(vals.max(), 4)
            })

df_resumen = pd.DataFrame(resumen_stats)
df_resumen.to_excel(output_dir / "resumen_estadistico_ntotal.xlsx", index=False)

# ============================================================================
# ANÁLISIS 5: GRÁFICO DE LÍNEAS - EVOLUCIÓN POR TIPO DE PROCESO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 5: Evolución del ECRPS por Proceso")
print("="*80)

for tipo_proceso in df['Tipo_Proceso'].unique():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    for modelo in MODELOS:
        df_filtrado = df[df['Tipo_Proceso'] == tipo_proceso]
        promedios = df_filtrado.groupby('N_Total')[modelo].mean()
        
        ax.plot(promedios.index, promedios.values, marker='o', linewidth=2, label=modelo)
    
    ax.set_xlabel('N_Total (Tamaño de Muestra)', fontsize=12)
    ax.set_ylabel('ECRPS Promedio', fontsize=12)
    ax.set_title(f'Evolución del ECRPS por N_Total\nTipo de Proceso: {tipo_proceso}',
                 fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10, bbox_to_anchor=(1.15, 1))
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    
    archivo_lineas = output_dir / f"evolucion_ecrps_{tipo_proceso.replace(' ', '_')}.png"
    plt.savefig(archivo_lineas, dpi=300, bbox_inches='tight')
    plt.close()

# ============================================================================
# ANÁLISIS 5B: TENDENCIA GLOBAL (TODOS LOS MODELOS EN UNA GRÁFICA)
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 5B: Tendencia Global de Modelos (ECRPS vs N_Total)")
print("="*80)

# Agrupar por N_Total para todos los procesos juntos (Promedio General)
fig, ax = plt.subplots(figsize=(16, 9))

colors = plt.cm.tab10(np.linspace(0, 1, len(MODELOS)))

for i, modelo in enumerate(MODELOS):
    # Calcular promedio global por N_Total (ignorando tipo de proceso)
    promedios_globales = df.groupby('N_Total')[modelo].mean()
    
    ax.plot(promedios_globales.index, promedios_globales.values, 
            marker='o', markersize=6, linewidth=2.5, 
            color=colors[i], label=modelo)

ax.set_xlabel('N_Total (Tamaño de Muestra)', fontsize=14)
ax.set_ylabel('ECRPS Promedio Global', fontsize=14)
ax.set_title('Tendencia Global: ECRPS Promedio por Modelo vs N_Total',
             fontsize=18, fontweight='bold', pad=20)

# Ajustar leyenda y grid
ax.legend(title='Modelos', title_fontsize=12, fontsize=11, 
          loc='upper left', bbox_to_anchor=(1, 1))
ax.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

archivo_global = output_dir / "tendencia_global_modelos.png"
plt.savefig(archivo_global, dpi=300, bbox_inches='tight')
print(f"Gráfica de tendencia global guardada: {archivo_global}")
plt.close()

# ============================================================================
# ANÁLISIS 6: RANKINGS Y FRECUENCIAS
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 6: Generando Rankings")
print("="*80)

rankings = []
for tipo_proceso in df['Tipo_Proceso'].unique():
    for ntotal in sorted(df['N_Total'].unique()):
        df_filtrado = df[(df['Tipo_Proceso'] == tipo_proceso) & (df['N_Total'] == ntotal)]
        
        errores_promedio = {}
        for modelo in MODELOS:
            errores_promedio[modelo] = df_filtrado[modelo].mean()
        
        ranking_modelos = sorted(errores_promedio.items(), key=lambda x: x[1])
        
        for rank, (modelo, score) in enumerate(ranking_modelos, 1):
            rankings.append({
                'Tipo_Proceso': tipo_proceso,
                'N_Total': ntotal,
                'Rank': rank,
                'Modelo': modelo,
                'ECRPS_Promedio': score
            })

df_rankings = pd.DataFrame(rankings)
archivo_rankings = output_dir / "rankings_modelos_por_ntotal.xlsx"
df_rankings.to_excel(archivo_rankings, index=False)

# ============================================================================
# ANÁLISIS 7: VARIABILIDAD (Heatmaps STD)
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 7: Variabilidad (Desviación Estándar)")
print("="*80)

variabilidad_resultados = []

# Calcular variabilidad general
for modelo in MODELOS:
    for ntotal in sorted(df['N_Total'].unique()):
        vals = df[df['N_Total'] == ntotal][modelo]
        variabilidad_resultados.append({
            'Escenario': 'General',
            'Modelo': modelo,
            'N_Total': ntotal,
            'Desv_Std': vals.std(),
            'CV': (vals.std() / vals.mean() * 100) if vals.mean() != 0 else 0
        })

df_variabilidad = pd.DataFrame(variabilidad_resultados)

# Generar Heatmap de Desviación Estándar (General)
df_esc = df_variabilidad[df_variabilidad['Escenario'] == 'General']
pivot_std = df_esc.pivot(index='Modelo', columns='N_Total', values='Desv_Std')

fig, ax = plt.subplots(figsize=(14, 10))

sns.heatmap(pivot_std, 
            annot=True, 
            fmt='.2f',  # <--- CAMBIO: Solo 2 dígitos
            cmap='YlOrRd',
            cbar_kws={'label': 'Desviación Estándar (ECRPS)', 'shrink': 0.6}, # <--- CAMBIO: Barra reducida
            linewidths=0.5,
            linecolor='gray',
            annot_kws={'size': 9},
            square=True,
            ax=ax)

plt.title('Variabilidad (Desviación Estándar) del ECRPS\nEscenario General',
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()

archivo_var = output_dir / "variabilidad_std_general.png"
plt.savefig(archivo_var, dpi=300, bbox_inches='tight')
print(f"Heatmap de variabilidad (STD) guardado: {archivo_var}")
plt.close()

# ============================================================================
# FINALIZAR
# ============================================================================
print("\n✅ Proceso completado. Revisa la carpeta de resultados.")
gc.collect()

Cargando datos...
Columnas disponibles: ['Paso', 'Proceso', 'Tipo_Proceso', 'Distribución', 'Varianza', 'N_Train', 'N_Calib', 'N_Total', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'ESCENARIO']
N_Total únicos: [np.int64(120), np.int64(140), np.int64(160), np.int64(200), np.int64(220), np.int64(240), np.int64(260), np.int64(300), np.int64(320), np.int64(340), np.int64(360), np.int64(400), np.int64(500), np.int64(520), np.int64(540), np.int64(560), np.int64(600), np.int64(700), np.int64(1020), np.int64(1040), np.int64(1060), np.int64(1100), np.int64(1200)]
Número de observaciones: 126000

ANÁLISIS 1: HEATMAPS - Promedio de ECRPS por Proceso y N_Total
Heatmap guardado: resultados_analisis_ntotal\heatmap_AREPD.png
Heatmap guardado: resultados_analisis_ntotal\heatmap_AV-MCPS.png
Heatmap guardado: resultados_analisis_ntotal\heatmap_Block_Bootstrapping.png
Heatmap guardado: resultados_analisis_ntotal\heatmap

8451